In [1]:
import numpy as np
import h5py
import cv2
import matplotlib.pyplot as plt
#from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
#from mpl_toolkits.mplot3d import Axes3D
# supress color warning when making movie
#from matplotlib.axes._axes import _log as matplotlib_axes_logger
#matplotlib_axes_logger.setLevel('ERROR')
%matplotlib widget
import time
%load_ext autoreload
import os
%autoreload 2
import sys
sys.path.append('./lib/')

from ipywidgets import Video, Image, VBox, Text
from sidecar import Sidecar
from IPython.display import display
from scipy.interpolate import interp1d
from scipy import interpolate
import numpy as np
import cv2
import base64

from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, Layout, AppLayout

import ipywidgets as widgets

#from bqplot import pyplot as plt

from DataManagement import SplitdataManager, Filepaths

In [2]:
# the path to the experiment
experiment_path = '/media/stephens-group/guest_drive/labelling/complete_shortfin_experiment/FishTank20200130_153857/'  
experiment_path

'/media/stephens-group/guest_drive/labelling/complete_shortfin_experiment/FishTank20200130_153857/'

In [3]:
# make a splitdata manager
splitman = SplitdataManager(experiment_path)

In [4]:
# gather the names of the splitdata folders with corresponding indices
splitdata_names = [triplet[0].split('/')[-1] for triplet in splitman.splitdata_paths]
splitdata_idxs = [i for i in range(len(splitdata_names))]
splitdata_name_to_idx_dict = dict(zip(splitdata_names, splitdata_idxs))
splitdata_name_to_paths = dict(zip(splitdata_names, splitman.splitdata_paths))
totalnumFrames = splitman.start_stop_frames_for_splitdata[-1, 1]


In [5]:
# load the trajectories from sLEAP tracked in the old way
trackTestPath = '/home/liam/Data/results_from_laetitia/FishTank20200130_153857/old_tracking_on_original_instances_sLEAP.h5'
with h5py.File(trackTestPath, 'r') as hf:
    tracks_3D_test = hf['tracks_3D'][:]
    
    
# load the new networks WITH tatsuos annotations for training, after conversion to original_instances format
inst_path = '/home/liam/Data/testing_network_accuracy_in_2_fish/original_instances_sLEAP.h5'
with h5py.File(inst_path, 'r') as hf:
    original_instances_new = hf['original_instances'][:]
    
# load the tracked image coordiantes that have gone through the calibration
with h5py.File('/home/liam/Data/results_from_laetitia/FishTank20200130_153857/old_tracking_on_original_instances_sLEAP_imCoords.h5', 'r') as hf:
    tracks_imCoords = hf['tracks_imCoords'][:]
    
# get the total number of frames in the experiment
totalnumFrames = tracks_3D_test.shape[0]

# other params
numFish = 2
numBodyPoints = 3
numCams = 3

# Making the Sidecar dashboard

In [6]:
## ---- Make the first panels of the dashboard --- ##

# make a dropdown menue to choose which splitdata you want
splitdata_chooser = widgets.Dropdown(options=splitdata_name_to_idx_dict, value=0)

# create and initialize widget to hold the splitdata_idx
splitdata_idx = widgets.IntText(value=0, description='splitdata_idx', layout=Layout(width='50%', height='50px'))
    
# create and initialize widget to hold the filepaths of the movies
xzMovPath =  widgets.Text(value=splitman.splitdata_paths[splitdata_idx.value][0] + '.mp4')
xyMovPath =  widgets.Text(value=splitman.splitdata_paths[splitdata_idx.value][1] + '.mp4')
yzMovPath =  widgets.Text(value=splitman.splitdata_paths[splitdata_idx.value][2] + '.mp4')

# create widgets to hold the global and local frame index
global_index  = widgets.IntText(value=0, description='Global fIdx', layout=Layout(width='70%', height='50px'))
local_index = widgets.IntText(value=0, description='Local fIdx', layout=Layout(width='70%', height='50px'))

# make a Global fIdx slide, and link it to the global frame number
global_index_slider = widgets.IntSlider(value=0, min=0, max=totalnumFrames-1, step=1, continuous_update=True,
                                       description='Frame', layout=Layout(width='90%', height='50px'))
gf_link = widgets.link((global_index_slider, 'value'), (global_index, 'value'))

    
def handle_splitdata_name_change(change):
    ''' Make required updates when we select a splitdata**** with the dropdown menu
    '''
    # update splitdata index
    splitdata_idx.value = change.new
    # update moviePaths
    xzMovPath.value = splitman.splitdata_paths[splitdata_idx.value][0] + '.mp4'
    xyMovPath.value = splitman.splitdata_paths[splitdata_idx.value][1] + '.mp4'
    yzMovPath.value = splitman.splitdata_paths[splitdata_idx.value][2] + '.mp4'
    # update the global index given the new splitdata index and the current local idx
    splitdata_start = splitman.start_stop_frames_for_splitdata[splitdata_idx.value][0]
    global_index.value = splitdata_start + local_index.value
# apply the function
splitdata_chooser.observe(handle_splitdata_name_change, names='value')



def handle_local_index_change(change):
    ''' Make the required changes when we update the "Local fIdx"
    '''
    # get the start-stop for the current splitdata index
    splitdata_start_stop = splitman.start_stop_frames_for_splitdata[splitdata_idx.value]
    splitdata_numFrames = splitdata_start_stop[1] - splitdata_start_stop[0]
    # find the new local index
    local_index.value = change.new
    # now make sure that the new index is within the range
    local_index.value = np.mod(local_index.value, splitdata_numFrames)
    # update global_frame
    global_index.value = splitdata_start_stop[0] + local_index.value
# apply the function
local_index.observe(handle_local_index_change, names='value')



def handle_global_index_change(change):
    ''' Make the required changes when we update the "Global fIdx"
    '''
    # get the start-stop for the current splitdata index
    splitdata_start_stop = splitman.start_stop_frames_for_splitdata[splitdata_idx.value]
    splitdata_numFrames = splitdata_start_stop[1] - splitdata_start_stop[0]
    # find the new global index
    if change.new >= totalnumFrames:
        global_index.value = totalnumFrames - 1
    else:
        global_index.value = change.new
    # get the current splitdata index and local index for this frame
    spIdx, locIdx = splitman.return_splitdata_folder_and_local_idx_for_global_frameIdx(global_index.value, return_splitdataIdx=True)
    local_index.value = locIdx
    splitdata_idx.value = spIdx
    splitdata_chooser.label = splitdata_names[spIdx]
    splitdata_chooser.index = spIdx
# apply the function
global_index.observe(handle_global_index_change, names='value')   
    
    
def handle_splitdata_idx_box_change(change):
    ''' Make the required changes when we update the "splitdata_idx" 
    '''
    # first, we must make sure that this new change is within the splitdatas available
    numSplitdatas = len(splitman.splitdata_paths)
    if change.new == -1:
        splitdata_idx.value = numSplitdatas - 1
    elif change.new == numSplitdatas:
        splitdata_idx.value = 0
    else:
        splitdata_idx.value = change.new
    # update the dropdown box
    splitdata_chooser.label = splitdata_names[splitdata_idx.value]
    splitdata_chooser.index = splitdata_idx.value
# apply
splitdata_idx.observe(handle_splitdata_idx_box_change, names='value')



# ---- make the final vertical box for sidecar and display --- ##

splitdata_details_box = VBox([splitdata_chooser, 
                              splitdata_idx, 
                              global_index, 
                              local_index, 
                              global_index_slider])



sc = Sidecar(title='Sidecar Output')
with sc:
    display(splitdata_details_box)
    

# Interactive plot of raw data in a range

In [7]:
tracks_3D = np.copy(tracks_3D_test)

In [8]:

plt.ioff()


# set the slider we are using
slider = global_index_slider

fig, (ax1, ax2) = plt.subplots(2, sharex=True, gridspec_kw={'hspace': 0.2})
fig.figsize = (18,10)
#fig.legend(handles, labels, loc='upper center')
fig.suptitle('Trajectory Data Timeseries')
fig.canvas.header_visible = False
fig.canvas.layout.min_height = '400px'
fig.canvas.layout.width = '100%'
ax1.grid(which='both')
ax2.grid(which='both')
ax1.ticklabel_format(useOffset=False)
ax2.ticklabel_format(useOffset=False)
plot_frame_width = 50

#plt.xticks(np.arange(0, totalnumFrames, 100))

# ---- define the info for each plot ----- #
# plot 1 - heads
bpIdx = 0
p1_data = np.copy(tracks_3D[:, :, bpIdx, :])

# plot 2 - pecs
bpIdx = 2
p2_data = np.copy(tracks_3D[:, :, bpIdx, :])

colors = ['lightcoral', 'brown', 'maroon', 
          'forestgreen', 'darkgreen', 'limegreen']


# ----- define the line plots ------ #
lines1_x1 = ax1.plot(p1_data[:, 0, 0], label='fish1_head_x', color=colors[0])
lines1_y1 = ax1.plot(p1_data[:, 0, 1], label='fish1_head_y', color=colors[1])
lines1_z1 = ax1.plot(p1_data[:, 0, 2], label='fish1_head_z', color=colors[2])

lines1_x2 = ax1.plot(p1_data[:, 1, 0], label='fish2_head_x', color=colors[3])
lines1_y2 = ax1.plot(p1_data[:, 1, 1], label='fish2_head_y', color=colors[4])
lines1_z2 = ax1.plot(p1_data[:, 1, 2], label='fish2_head_z', color=colors[5])



lines2_x1 = ax2.plot(p2_data[:, 0, 0], label='fish1_x', color=colors[0])
lines2_y1 = ax2.plot(p2_data[:, 0, 1], label='fish1_y', color=colors[1])
lines2_z1 = ax2.plot(p2_data[:, 0, 2], label='fish1_z', color=colors[2])

lines2_x2 = ax2.plot(p2_data[:, 1, 0], label='fish2_x', color=colors[3])
lines2_y2 = ax2.plot(p2_data[:, 1, 1], label='fish2_y', color=colors[4])
lines2_z2 = ax2.plot(p2_data[:, 1, 2], label='fish2_z', color=colors[5])


#ax1.legend(loc='upper right')
#ax2.legend(loc='upper right')


#lines2 = ax2.plot(p2_data)

ax1.set_title('Heads: frame = {0}'.format(slider.value))
ax2.set_title('Tails)')
#ax2.set_title('Plotting: global_fIdx = {0})'.format(slider.value))

ax1.set_ylim(-1, 43)
ax2.set_ylim(-1, 43)

ax1.set_yticks([i for i in range(0,41,5)], minor=True)
ax2.set_yticks([i for i in range(0,41,5)], minor=True)


#ax2.set_yticklabels([i for i in range(0, 40, 5)])

ax1.set_ylabel('Head Position (cm)')
ax2.set_ylabel('Tail Position (cm)')

# use any axis here to get the labels
handles, labels = ax2.get_legend_handles_labels()
fig.legend(handles, labels, loc='center right')

if slider.value < plot_frame_width:
    ax1.set_xlim(0, 2*plot_frame_width)
    ax2.set_xlim(0, 2*plot_frame_width)
elif totalnumFrames - slider.value < plot_frame_width:
    ax1.set_xlim(totalnumFrames-2*plot_frame_width, totalnumFrames)
    ax2.set_xlim(totalnumFrames-2*plot_frame_width, totalnumFrames)
else:
    ax1.set_xlim(slider.value-plot_frame_width, slider.value+plot_frame_width)
    ax2.set_xlim(slider.value-plot_frame_width, slider.value+plot_frame_width)



def update_lines(change):
    time.sleep(0.01)
    ax1.set_title('Heads: frame = {0})'.format(change.new))
    ax2.set_title('Tails')
    #ax2.set_title('Plotting: global_fIdx = {0})'.format(change.new))
    if change.new < plot_frame_width:
        ax1.set_xlim(0, 2*plot_frame_width)
        ax2.set_xlim(0, 2*plot_frame_width)
    elif totalnumFrames - change.new < plot_frame_width:
        ax1.set_xlim(totalnumFrames-2*plot_frame_width, totalnumFrames)
        ax2.set_xlim(totalnumFrames-2*plot_frame_width, totalnumFrames)
    else:
        ax1.set_xlim(slider.value-plot_frame_width, slider.value+plot_frame_width)
        ax2.set_xlim(slider.value-plot_frame_width, slider.value+plot_frame_width)
    #lines[0].set_data(timearr, data)
    #plt.xlim(timearr[0], timearr[-1])
    #fig.clf()
    fig.canvas.draw()
    fig.canvas.flush_events()
    
slider.observe(update_lines, names='value')

layout = widgets.Layout(width='100%') #set width and height

aaa= VBox([fig.canvas, slider], layout=layout)

# aaa= AppLayout(
#     center=fig.canvas,
#     footer=slider)
#)

aaa

# Clicking on images

In [9]:
## ---- Interpolate the heads for making movies without gaps ----- ##

def fill_nan(A):
    '''
    Thanks - https://stackoverflow.com/a/9815522
    interpolate to fill nan values
    '''
    inds = np.arange(A.shape[0])
    good = np.where(np.isfinite(A))
    f = interpolate.interp1d(inds[good], A[good],bounds_error=False)
    B = np.where(np.isfinite(A),A,f(inds))
    return B

# interpolate the fish_pecs so we dont blink
xz_heads = np.copy(original_instances_new[0,:,:,0,:])
xy_heads = np.copy(original_instances_new[1,:,:,0,:])
yz_heads = np.copy(original_instances_new[2,:,:,0,:])

xz_heads_interpd = np.copy(xz_heads)
xy_heads_interpd = np.copy(xy_heads)
yz_heads_interpd = np.copy(yz_heads)

numFish=2
for fishIdx in range(numFish):
    
    xz_heads_x = np.copy(xz_heads[:, fishIdx, 0])
    xz_heads_y = np.copy(xz_heads[:, fishIdx, 1])
    xz_heads_x_interpd = fill_nan(xz_heads_x)
    xz_heads_y_interpd = fill_nan(xz_heads_y)
    xz_heads_interpd[:, fishIdx, 0] = xz_heads_x_interpd
    xz_heads_interpd[:, fishIdx, 1] = xz_heads_y_interpd
    
    xy_heads_x = np.copy(xy_heads[:, fishIdx, 0])
    xy_heads_y = np.copy(xy_heads[:, fishIdx, 1])
    xy_heads_x_interpd = fill_nan(xy_heads_x)
    xy_heads_y_interpd = fill_nan(xy_heads_y)
    xy_heads_interpd[:, fishIdx, 0] = xy_heads_x_interpd
    xy_heads_interpd[:, fishIdx, 1] = xy_heads_y_interpd
    
    yz_heads_x = np.copy(yz_heads[:, fishIdx, 0])
    yz_heads_y = np.copy(yz_heads[:, fishIdx, 1])
    yz_heads_x_interpd = fill_nan(yz_heads_x)
    yz_heads_y_interpd = fill_nan(yz_heads_y)
    yz_heads_interpd[:, fishIdx, 0] = yz_heads_x_interpd
    yz_heads_interpd[:, fishIdx, 1] = yz_heads_y_interpd
    
    
# make a list of crop_centers for the frames
crop_centers = [xz_heads_interpd, xy_heads_interpd, yz_heads_interpd]

In [77]:
# ----- Grab the image coordinates and 3D coordinats for this frame ---- #
number_layout = widgets.Layout(width='70px')
text_layout = widgets.Layout(width='120px')

# default positions of bodypoints in cropped coords
p1_init = np.array([20, 20])
p2_init = np.array([80, 80])
p3_init = np.array([140, 140])
def_cropped_pos = [p1_init, p2_init, p3_init]

# preallocate list structures to hold the numbers
frame_imCoords_widget_list = [[[[ [] for _ in range(2)] for _ in range(numBodyPoints)] for _ in range(numFish)] for _ in range(numCams)]
frame_3D_widget_list = [ [[[] for _ in range(3)] for _ in range(numBodyPoints)] for _ in range(numFish)]
# fill in the widgets with initial values
for camIdx in range(numCams):
    for fishIdx in range(numFish):
        for bpIdx in range(numBodyPoints):
            for imCoordIdx in range(2):
                wid_val = np.round(np.copy(tracks_imCoords[camIdx, global_index_slider.value, fishIdx, bpIdx, imCoordIdx]),2)
                val_widge_coord = widgets.FloatText(value=wid_val,layout=number_layout)
                frame_imCoords_widget_list[camIdx][fishIdx][bpIdx][imCoordIdx] = val_widge_coord
                
for fishIdx in range(numFish):
    for bpIdx in range(numBodyPoints):
        for dimIdx in range(3):
            wid_val = np.round(np.copy(tracks_3D[global_index_slider.value, fishIdx, bpIdx, dimIdx]), 2)
            val_widge_3D = widgets.FloatText(value=wid_val,layout=number_layout)
            frame_3D_widget_list[fishIdx][bpIdx][dimIdx] = val_widge_3D


In [78]:
# ---- Make widgets to vizualize the 3D positions --- #
number_layout = widgets.Layout(width='80px')
text_layout = widgets.Layout(width='120px')

fishIdx = 0
f1_bp_box_list = []
for bpIdx in range(numBodyPoints):
    f1_bp_XYZ_box = widgets.HBox(frame_3D_widget_list[fishIdx][bpIdx][:])
    f1_bp_box_list.append(f1_bp_XYZ_box)
f1_3D_wid_title = widgets.Text('Fish1 3D bps', layout=text_layout)
f1_3D_wid = widgets.VBox([f1_3D_wid_title]+f1_bp_box_list)


fishIdx = 1
f2_bp_box_list = []
for bpIdx in range(numBodyPoints):
    f2_bp_XYZ_box = widgets.HBox(frame_3D_widget_list[fishIdx][bpIdx][:])
    f2_bp_box_list.append(f2_bp_XYZ_box)
f2_3D_wid_title = widgets.Text('Fish2 3D bps', layout=text_layout)
f2_3D_wid = widgets.VBox([f2_3D_wid_title]+f2_bp_box_list)

In [79]:
f1_3D_wid

In [80]:
f2_3D_wid

In [95]:
# ---- Make widgets to vizualize the image coordinates --- #
number_layout = widgets.Layout(width='80px')
text_layout = widgets.Layout(width='120px')
cam_title_layout = widgets.Layout(width='160px')
title_layout = widgets.Layout(width='480px')
camNames = ['xz', 'xy', 'yz']

fishIdx = 0
cam_bp_imcoord_wid_list = []
for camIdx in range(3):
    cam_bp_list = []
    for bpIdx in range(numBodyPoints):
        bp_imcoords_box = widgets.HBox(frame_imCoords_widget_list[camIdx][fishIdx][bpIdx][:])
        cam_bp_list.append(bp_imcoords_box)
    cam_widget_title = widgets.Text(camNames[camIdx], layout=cam_title_layout)
    cam_widget = widgets.VBox([cam_widget_title]+cam_bp_list)
    cam_bp_imcoord_wid_list.append(cam_widget)
f1_imcoord_wid_title = widgets.Text('Fish1 bodypoint image coordinates', layout=title_layout)
f1_imcoord_wid_data = widgets.HBox(cam_bp_imcoord_wid_list)
f1_imcoord_wid = widgets.VBox([f1_imcoord_wid_title, f1_imcoord_wid_data])


fishIdx = 1
cam_bp_imcoord_wid_list = []
for camIdx in range(3):
    cam_bp_list = []
    for bpIdx in range(numBodyPoints):
        bp_imcoords_box = widgets.HBox(frame_imCoords_widget_list[camIdx][fishIdx][bpIdx][:])
        cam_bp_list.append(bp_imcoords_box)
    cam_widget_title = widgets.Text(camNames[camIdx], layout=cam_title_layout)
    cam_widget = widgets.VBox([cam_widget_title]+cam_bp_list)
    cam_bp_imcoord_wid_list.append(cam_widget)
f2_imcoord_wid_title = widgets.Text('Fish1 bodypoint image coordinates', layout=title_layout)
f2_imcoord_wid_data = widgets.HBox(cam_bp_imcoord_wid_list)
f2_imcoord_wid = widgets.VBox([f2_imcoord_wid_title, f2_imcoord_wid_data])





In [94]:
f1_imcoord_wid

In [96]:
f2_imcoord_wid

In [34]:
frame_imCoords_widget_list

[[[[FloatText(value=778.8842758771268), FloatText(value=929.3468749481663)],
   [FloatText(value=769.7296424847586), FloatText(value=929.527265568973)],
   [FloatText(value=757.3853040298593), FloatText(value=930.6809011357515)]],
  [[FloatText(value=845.3420474577254), FloatText(value=876.4227213613357)],
   [FloatText(value=827.528092826262), FloatText(value=884.4767874787433)],
   [FloatText(value=796.1503010598362), FloatText(value=908.601120819165)]]],
 [[[FloatText(value=523.5456968048638), FloatText(value=314.4798570184764)],
   [FloatText(value=530.9429135178773), FloatText(value=299.80890437270284)],
   [FloatText(value=541.6787704102287), FloatText(value=257.6910704685283)]],
  [[FloatText(value=468.3325733966135), FloatText(value=383.61628254965336)],
   [FloatText(value=482.5945985601861), FloatText(value=377.0845471297598)],
   [FloatText(value=508.2532834876103), FloatText(value=354.4837183155365)]]],
 [[[FloatText(value=869.3193012312945), FloatText(value=879.21828250739

In [35]:
frame_3D_widget_list

[[[FloatText(value=14.283960178450798),
   FloatText(value=30.46170675321173),
   FloatText(value=0.4841095111469258)],
  [FloatText(value=14.674729559792468),
   FloatText(value=31.248256911336284),
   FloatText(value=0.5998305933417214)],
  [FloatText(value=15.233755690382168),
   FloatText(value=33.50759101053336),
   FloatText(value=0.8946740998867426)]],
 [[FloatText(value=11.477508208447034),
   FloatText(value=26.643974330986566),
   FloatText(value=2.1388383216326474)],
  [FloatText(value=12.21814707415615),
   FloatText(value=27.00781225810715),
   FloatText(value=1.8556647931172847)],
  [FloatText(value=13.527465400065921),
   FloatText(value=28.26792999547888),
   FloatText(value=1.0247494363770784)]]]

In [36]:
tracks_3D[global_index_slider.value]

array([[[14.28396018, 30.46170675,  0.48410951],
        [14.67472956, 31.24825691,  0.59983059],
        [15.23375569, 33.50759101,  0.8946741 ]],

       [[11.47750821, 26.64397433,  2.13883832],
        [12.21814707, 27.00781226,  1.85566479],
        [13.5274654 , 28.26793   ,  1.02474944]]])

In [14]:
def _pad_frame(frame, padding_depth):
    ''' Return a padded version of the frame '''
    # grab the padding from the image
    padding = frame[0:padding_depth,:]
    # make the padded frame
    frame = np.vstack((padding, frame, padding))
    return frame

# make a list of colors
no_id_color = (255,255,255) # white
edges = [[0,1], [1,2]]
padding_depth = 70
bw = 20
cropSize = (80,80)

plt.ioff()

fig1, (f1_ax1, f1_ax2, f1_ax3) = plt.subplots(1, 3, figsize=(9,3))
fig2, (f2_ax1, f2_ax2, f2_ax3) = plt.subplots(1, 3, figsize=(9,3))

f1_axs = [f1_ax1, f1_ax2, f1_ax3]
f2_axs = [f2_ax1, f2_ax2, f2_ax3]
f_axs = [f1_axs, f2_axs]


f1_ax1.set_title('Fish1_xz')
f1_ax2.set_title('Fish1_xy')
f1_ax3.set_title('Fish1_yz')

f2_ax1.set_title('Fish2_xz')
f2_ax2.set_title('Fish2_xy')
f2_ax3.set_title('Fish2_yz')

fig1.canvas.header_visible = False
fig1.canvas.layout.min_height = '400px'
fig1.canvas.layout.width = '100%'

fig2.canvas.header_visible = False
fig2.canvas.layout.min_height = '400px'
fig2.canvas.layout.width = '100%'


#  -------------- get the initial frame data -------------------#
video_paths = [xzMovPath, xyMovPath, yzMovPath]
caps = [cv2.VideoCapture(video_path.value) for video_path in video_paths]
[cap.set(1,local_index.value) for cap in caps] #go to current frame
# grab the frame from each camera
if caps[0].isOpened() and caps[1].isOpened() and caps[2].isOpened():  
    # grab the 3 images
    ret_xz, frame_xz = caps[0].read()
    if not ret_xz:
        raise TypeError('xz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
    ret_xy, frame_xy = caps[1].read()
    if not ret_xy:
        raise TypeError('xy movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
    ret_yz, frame_yz = caps[2].read()
    if not ret_yz:
        raise TypeError('yz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
    frames = [frame_xz, frame_xy, frame_yz]
# close everything before we finish
[caps[capIdx].release() for capIdx in range(3)];




# ------ plot the images --------- #

f1_panel1 = f1_ax1.imshow(frames[0])
f1_panel2 = f1_ax2.imshow(frames[1])
f1_panel3 = f1_ax3.imshow(frames[2])
f1_panels = [f1_panel1, f1_panel2, f1_panel3]

f2_panel1 = f2_ax1.imshow(frames[0])
f2_panel2 = f2_ax2.imshow(frames[1])
f2_panel3 = f2_ax3.imshow(frames[2])
f2_panels = [f2_panel1, f2_panel2, f2_panel3]

f_panels = [f1_panels, f2_panels]


# set the images limits (the default zoom)
imshow_ranges = [[ [] for _ in range(numCams)] for _ in range(numFish)]
for fishIdx in range(numFish):
    for camIdx in range(3):
        cc = crop_centers[camIdx][global_index_slider.value, fishIdx, :]
        if ~np.all(np.isnan(cc)):
            f_axs[fishIdx][camIdx].set_xlim(cc[0]-int(cropSize[0]), cc[0]+int(cropSize[0]))
            f_axs[fishIdx][camIdx].set_ylim(cc[1]-int(cropSize[1]), cc[1]+int(cropSize[1]))




# # plot the default positions
# for fishIdx in range(numFish):
#     for camIdx in range(3):
#         xs = []
#         ys = []
#         for bpIdx in range(numBodyPoints):
#             x = frame_imCoords_widget_list[camIdx][fishIdx][bpIdx][0].value
#             y = frame_imCoords_widget_list[camIdx][fishIdx][bpIdx][1].value
#             xs.append(x)
#             ys.append(y)
#         # draw the bodypoints
#         f_axs[fishIdx][camIdx].scatter(xs,ys,s=8, color='blue')
#         # draw the lines
#         f_axs[fishIdx][camIdx].plot(xs,ys, color='blue')


        
        
def update_crop_figures(change):
    caps = [cv2.VideoCapture(video_path.value) for video_path in video_paths]
    [cap.set(1,local_index.value) for cap in caps] #go to current frame
    # grab the frame from each camera
    if caps[0].isOpened() and caps[1].isOpened() and caps[2].isOpened():  
        # grab the 3 images
        ret_xz, frame_xz = caps[0].read()
        if not ret_xz:
            raise TypeError('xz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
        ret_xy, frame_xy = caps[1].read()
        if not ret_xy:
            raise TypeError('xy movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
        ret_yz, frame_yz = caps[2].read()
        if not ret_yz:
            raise TypeError('yz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
        frames = [frame_xz, frame_xy, frame_yz]
    # release the caps
    [caps[capIdx].release() for capIdx in range(3)];
            
    # now update the figures
    f1_panel1.set_data(frames[0])
    f1_panel2.set_data(frames[1])
    f1_panel3.set_data(frames[2])
    
    f2_panel1.set_data(frames[0])
    f2_panel2.set_data(frames[1])
    f2_panel3.set_data(frames[2])
    
    
    # set the images limits (the default zoom)
    imshow_ranges = [[ [] for _ in range(numCams)] for _ in range(numFish)]
    for fishIdx in range(numFish):
        for camIdx in range(3):
            cc = crop_centers[camIdx][global_index_slider.value, fishIdx, :]
            if ~np.all(np.isnan(cc)):
                f_axs[fishIdx][camIdx].set_xlim(cc[0]-int(cropSize[0]), cc[0]+int(cropSize[0]))
                f_axs[fishIdx][camIdx].set_ylim(cc[1]-int(cropSize[1]), cc[1]+int(cropSize[1]))
    
    fig1.canvas.draw()
    fig1.canvas.flush_events()
    fig2.canvas.draw()
    fig2.canvas.flush_events()
    return
        
        
        
global_index_slider.observe(update_crop_figures, names='value')


bbb = widgets.VBox([fig1.canvas, fig2.canvas])
bbb

In [ ]:
# ---------------------------------------------------------------------------------------------------------- #

In [13]:
def _pad_frame(frame, padding_depth):
    ''' Return a padded version of the frame '''
    # grab the padding from the image
    padding = frame[0:padding_depth,:]
    # make the padded frame
    frame = np.vstack((padding, frame, padding))
    return frame

# make a list of colors
no_id_color = (255,255,255) # white
edges = [[0,1], [1,2]]
padding_depth = 70
bw = 20
cropSize = (80,80)

plt.ioff()

fig1, (f1_ax1, f1_ax2, f1_ax3) = plt.subplots(1, 3, figsize=(9,3))
fig2, (f2_ax1, f2_ax2, f2_ax3) = plt.subplots(1, 3, figsize=(9,3))

f1_axs = [f1_ax1, f1_ax2, f1_ax3]
f2_axs = [f2_ax1, f2_ax2, f2_ax3]
f_axs = [f1_axs, f2_axs]


f1_ax1.set_title('Fish1_xz')
f1_ax2.set_title('Fish1_xy')
f1_ax3.set_title('Fish1_yz')

f2_ax1.set_title('Fish2_xz')
f2_ax2.set_title('Fish2_xy')
f2_ax3.set_title('Fish2_yz')

fig1.canvas.header_visible = False
fig1.canvas.layout.min_height = '400px'
fig1.canvas.layout.width = '100%'

fig2.canvas.header_visible = False
fig2.canvas.layout.min_height = '400px'
fig2.canvas.layout.width = '100%'


#  -------------- get the initial frame data -------------------#
video_paths = [xzMovPath, xyMovPath, yzMovPath]
caps = [cv2.VideoCapture(video_path.value) for video_path in video_paths]
[cap.set(1,local_index.value) for cap in caps] #go to current frame
# grab the frame from each camera
if caps[0].isOpened() and caps[1].isOpened() and caps[2].isOpened():  
    # grab the 3 images
    ret_xz, frame_xz = caps[0].read()
    if not ret_xz:
        raise TypeError('xz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
    ret_xy, frame_xy = caps[1].read()
    if not ret_xy:
        raise TypeError('xy movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
    ret_yz, frame_yz = caps[2].read()
    if not ret_yz:
        raise TypeError('yz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
    frames = [frame_xz, frame_xy, frame_yz]
# take crops around fish1 and fish2
crops_fish1 = []
crops_fish2 = []
for camIdx in range(3):
    fish1_head = crop_centers[camIdx][global_index_slider.value, 0]
    fish2_head = crop_centers[camIdx][global_index_slider.value, 1]
    # get the center of the crop, adjusted for padding
    cropCenter1 = [fish1_head[0], fish1_head[1]+padding_depth]
    cropCenter2 = [fish2_head[0], fish2_head[1]+padding_depth]

    if np.all(np.isnan(fish1_head)):
        crop = np.zeros((cropSize[0]*2, cropSize[1]*2, 3), dtype=np.uint8)
        crops_fish1.append(crop)
    else:
        frame = frames[camIdx]
        frame = _pad_frame(frame, padding_depth)
        crop = frame[int(cropCenter1[1]-cropSize[1]):int(cropCenter1[1]+cropSize[1]),
                     int(cropCenter1[0]-cropSize[0]):int(cropCenter1[0]+cropSize[0])]
        crops_fish1.append(crop)

    if np.all(np.isnan(fish2_head)):
        crop = np.zeros((cropSize[0]*2, cropSize[1]*2, 3), dtype=np.uint8)
        crops_fish2.append(crop)
    else:
        frame = frames[camIdx]
        frame = _pad_frame(frame, padding_depth)
        crop = frame[int(cropCenter2[1]-cropSize[1]):int(cropCenter2[1]+cropSize[1]),
                     int(cropCenter2[0]-cropSize[0]):int(cropCenter2[0]+cropSize[0])]
        crops_fish2.append(crop)
# close everything before we finish
[caps[capIdx].release() for capIdx in range(3)];




# plot the images
f1_panel1 = f1_ax1.imshow(crops_fish1[0])
f1_panel2 = f1_ax2.imshow(crops_fish1[1])
f1_panel3 = f1_ax3.imshow(crops_fish1[2])

f2_panel1 = f2_ax1.imshow(crops_fish2[0])
f2_panel2 = f2_ax2.imshow(crops_fish2[1])
f2_panel3 = f2_ax3.imshow(crops_fish2[2])

# plot the default positions
for fishIdx in range(numFish):
    for camIdx in range(3):
        xs = []
        ys = []
        for bpIdx in range(numBodyPoints):
            x = frame_imCoords_widget_list[camIdx][fishIdx][bpIdx][0].value
            y = frame_imCoords_widget_list[camIdx][fishIdx][bpIdx][1].value
            xs.append(x)
            ys.append(y)
        # draw the bodypoints
        f_axs[fishIdx][camIdx].scatter(xs,ys,s=8, color='blue')
        # draw the lines
        f_axs[fishIdx][camIdx].plot(xs,ys, color='blue')


        
        
def update_crop_figures(change):
    caps = [cv2.VideoCapture(video_path.value) for video_path in video_paths]
    [cap.set(1,local_index.value) for cap in caps] #go to current frame
    # grab the frame from each camera
    if caps[0].isOpened() and caps[1].isOpened() and caps[2].isOpened():  
        # grab the 3 images
        ret_xz, frame_xz = caps[0].read()
        if not ret_xz:
            raise TypeError('xz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
        ret_xy, frame_xy = caps[1].read()
        if not ret_xy:
            raise TypeError('xy movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
        ret_yz, frame_yz = caps[2].read()
        if not ret_yz:
            raise TypeError('yz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
        frames = [frame_xz, frame_xy, frame_yz]
    # take crops around fish1 and fish2
    crops_fish1 = []
    crops_fish2 = []
    for camIdx in range(3):
        fish1_head = crop_centers[camIdx][change.new, 0]
        fish2_head = crop_centers[camIdx][change.new, 1]
        # get the center of the crop, adjusted for padding
        cropCenter1 = [fish1_head[0], fish1_head[1]+padding_depth]
        cropCenter2 = [fish2_head[0], fish2_head[1]+padding_depth]

        if np.all(np.isnan(fish1_head)):
            crop = np.zeros((cropSize[0]*2, cropSize[1]*2, 3), dtype=np.uint8)
            crops_fish1.append(crop)
        else:
            frame = frames[camIdx]
            frame = _pad_frame(frame, padding_depth)
            crop = frame[int(cropCenter1[1]-cropSize[1]):int(cropCenter1[1]+cropSize[1]),
                         int(cropCenter1[0]-cropSize[0]):int(cropCenter1[0]+cropSize[0])]
            crops_fish1.append(crop)

        if np.all(np.isnan(fish2_head)):
            crop = np.zeros((cropSize[0]*2, cropSize[1]*2, 3), dtype=np.uint8)
            crops_fish2.append(crop)
        else:
            frame = frames[camIdx]
            frame = _pad_frame(frame, padding_depth)
            crop = frame[int(cropCenter2[1]-cropSize[1]):int(cropCenter2[1]+cropSize[1]),
                         int(cropCenter2[0]-cropSize[0]):int(cropCenter2[0]+cropSize[0])]
            crops_fish2.append(crop)
            
    # release the caps
    [caps[capIdx].release() for capIdx in range(3)];
            
    # now update the figures
    f1_panel1.set_data(crops_fish1[0])
    f1_panel2.set_data(crops_fish1[1])
    f1_panel3.set_data(crops_fish1[2])
    
    f2_panel1.set_data(crops_fish2[0])
    f2_panel2.set_data(crops_fish2[1])
    f2_panel3.set_data(crops_fish2[2])

    fig1.canvas.draw()
    fig1.canvas.flush_events()
    fig2.canvas.draw()
    fig2.canvas.flush_events()
    return
        
        
        
global_index_slider.observe(update_crop_figures, names='value')


bbb = widgets.VBox([fig1.canvas, fig2.canvas])
bbb

In [ ]:
def convert_global_coord_to_crop_coord(imCoord_gbl, cropCenter, cropShape=(80,80)):
    ''' Return the image coordinates of imCoord_gbl in the coordinate system of the crop
        centered at cropCenter
    '''

In [19]:
camIdx = 0
fishIdx = 0
cropCenter = crop_centers[camIdx][global_index_slider.value, fishIdx]
cropCenter

array([881.99645996, 850.04840088])

In [23]:
tracks_imCoords[camIdx, global_index_slider.value, fishIdx, :]

array([[963.58823176, 841.94716902],
       [959.55620835, 841.09216092],
       [960.29031111, 841.15286664]])

In [ ]:
tlc = crop.

In [11]:
frame_imCoords_numParams = numCams*numFish*numBodyPoints*2
frame_3D_numParams = numFish*numBodyPoints*3

# preallocate list structures to hold the numbers
frame_imCoords_widget_list = [[[[ [] for _ in range(2)] for _ in range(numBodyPoints)] for _ in range(numFish)] for _ in range(numCams)]
frame_3D_widget_list = [ [[[] for _ in range(3)] for _ in range(numBodyPoints)] for _ in range(numFish)]

# fill in the widgets with initial values
for camIdx in range(numCams):
    for fishIdx in range(numFish):
        for bpIdx in range(numBodyPoints):
            for imCoordIdx in range(2):
                val_widge = widgets.FloatText(value=np.copy(tracks_imCoords[camIdx, global_index_slider.value, fishIdx, bpIdx, imCoordIdx]))
                frame_imCoords_widget_list[camIdx][fishIdx][bpIdx][imCoordIdx] = val_widge
                
for fishIdx in range(numFish):
    for bpIdx in range(numBodyPoints):
        for dimIdx in range(3):
            val_widge = widgets.FloatText(value=np.copy(tracks_3D[global_index_slider.value, fishIdx, bpIdx, dimIdx]))
            frame_3D_widget_list[fishIdx][bpIdx][dimIdx] = val_widge



In [12]:
frame_imCoords_widget_list

[[[[FloatText(value=963.588231764288), FloatText(value=841.9471690157883)],
   [FloatText(value=959.5562083540699), FloatText(value=841.0921609151302)],
   [FloatText(value=960.2903111087325), FloatText(value=841.1528666370532)]],
  [[FloatText(value=881.5407670820146), FloatText(value=843.6816831138434)],
   [FloatText(value=886.35452823585), FloatText(value=846.7056352695242)],
   [FloatText(value=901.033695712988), FloatText(value=851.0676178688509)]]],
 [[[FloatText(value=351.9815005991361), FloatText(value=630.4149200345162)],
   [FloatText(value=356.40563595226524), FloatText(value=618.9409875330883)],
   [FloatText(value=359.44210597067547), FloatText(value=580.1701228295066)]],
  [[FloatText(value=416.22083634561), FloatText(value=701.663847008464)],
   [FloatText(value=413.3583471415184), FloatText(value=684.7478647520197)],
   [FloatText(value=403.4632795475758), FloatText(value=651.071894568366)]]],
 [[[FloatText(value=493.9735449016592), FloatText(value=831.1818537892755)],

In [13]:
frame_3D_widget_list

[[[FloatText(value=5.422331177394881),
   FloatText(value=13.41747463089905),
   FloatText(value=1.7859048617542612)],
  [FloatText(value=5.669096065572376),
   FloatText(value=14.04604155836114),
   FloatText(value=1.9165117876382283)],
  [FloatText(value=5.831462466025187),
   FloatText(value=16.149705147823624),
   FloatText(value=2.1959980939100205)]],
 [[FloatText(value=8.910768068128164),
   FloatText(value=9.51889813735761),
   FloatText(value=1.2568916867285225)],
  [FloatText(value=8.749659035311197),
   FloatText(value=10.444886651511297),
   FloatText(value=1.2314617080769494)],
  [FloatText(value=8.199924970258508),
   FloatText(value=12.28406020933744),
   FloatText(value=1.2599254785291123)]]]

In [14]:
p1_init = np.array([20, 20])
p2_init = np.array([80, 80])
p3_init = np.array([140, 140])
ps_init = [p1_init, p2_init, p3_init]

old_wid_pos = [ [[] for _ in range(2)] for _ in range(len(ps_init))] 
for i in range(len(ps_init)):
    for j in range(2):
        val = ps_init[i][j]
        old_wid_pos[i][j] = widgets.FloatText(value=val)
        

new_wid_pos = [ [[] for _ in range(2)] for _ in range(len(ps_init))] 
for i in range(len(ps_init)):
    for j in range(2):
        val = ps_init[i][j]
        new_wid_pos[i][j] = widgets.FloatText(value=val)


# old_widget_values = [p1_init, p2_init, p3_init]

In [15]:
old_wid_pos

[[FloatText(value=20.0), FloatText(value=20.0)],
 [FloatText(value=80.0), FloatText(value=80.0)],
 [FloatText(value=140.0), FloatText(value=140.0)]]

In [16]:
new_wid_pos

[[FloatText(value=20.0), FloatText(value=20.0)],
 [FloatText(value=80.0), FloatText(value=80.0)],
 [FloatText(value=140.0), FloatText(value=140.0)]]

In [17]:
def find_idx_of_closest_point(x, y, tolerance=15):
    ''' Return the bodypoint corresponding to this click
    '''
    distances = []
    for pointIdx in range(3):
        xp = old_wid_pos[pointIdx][0].value
        yp = old_wid_pos[pointIdx][1].value
        dist = (((x-xp)**2) + ((y-yp)**2))**0.5
        distances.append(dist)
    closest_point_idx = np.argmin(distances)
    if distances[closest_point_idx] < tolerance:
        bpIdx.value = closest_point_idx
    else:
        bpIdx.value = -1
    

In [ ]:
def find_idx_of_closest_point(x, y, tolerance=15):
    ''' Return the bodypoint corresponding to this click
    '''
    distances = []
    for pointIdx in range(3):
        xp = old_wid_pos[pointIdx][0].value
        yp = old_wid_pos[pointIdx][1].value
        dist = (((x-xp)**2) + ((y-yp)**2))**0.5
        distances.append(dist)
    closest_point_idx = np.argmin(distances)
    if distances[closest_point_idx] < tolerance:
        bpIdx.value = closest_point_idx
    else:
        bpIdx.value = -1
        
    
def click_down_event_handler(event): 
    # find the closest point to old_wis_pos
    bpIdx.value = find_idx_of_closest_point(event.xdata, event.ydata, tolerance=15)
#     for pIdx in range(len(old_wid_pos)):
#         x = old_wid_pos[pIdx][0].value
#         y = old_wid_pos[pIdx][1].value
    ax.scatter(60,60,s=8, color='blue')
    fig.canvas.draw()
    fig.canvas.flush_events()
    
    
fig.canvas.mpl_connect('button_press_event', click_down_event_handler)

In [46]:
test_image = np.copy(crops_fish1[0])


#plt.ion()
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(test_image)
ax.scatter(30,30,s=8,color='r')

bpIdx = widgets.IntText(value=0)

def click_down_event_handler(event): 
    # find the closest point to old_wis_pos
    bpIdx.value = find_idx_of_closest_point(event.xdata, event.ydata, tolerance=15)
#     for pIdx in range(len(old_wid_pos)):
#         x = old_wid_pos[pIdx][0].value
#         y = old_wid_pos[pIdx][1].value
    ax.scatter(60,60,s=8, color='blue')
    fig.canvas.draw()
    fig.canvas.flush_events()
    

fig.canvas.mpl_connect('button_press_event', click_down_event_handler)

ppp = widgets.VBox([fig.canvas])
ppp

#cid1 = fig.canvas.mpl_connect('button_press_event', onclick)

/home/liam/anaconda3/envs/FishTank/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


In [47]:
bpIdx

IntText(value=0)

In [21]:
test_image = np.copy(crops_fish1[0])

old_coord_widget_x = widgets.FloatText(value=0)
old_coord_widget_y = widgets.FloatText(value=0)

new_coord_widget_x = widgets.FloatText(value=0)
new_coord_widget_y = widgets.FloatText(value=0)


#plt.ion()
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(test_image)

coords = []

def click_down_event_handler(event):
    old_coord_widget_x.value = new_coord_widget_x.value   
    old_coord_widget_y.value = new_coord_widget_y.value  
    return
    
def click_release_event_handler(event):
    new_coord_widget_x.value = event.xdata 
    new_coord_widget_y.value = event.ydata 
    return 

fig.canvas.mpl_connect('button_press_event', click_down_event_handler)
fig.canvas.mpl_connect('button_release_event', click_release_event_handler)

ppp = widgets.VBox([fig.canvas])
ppp

#cid1 = fig.canvas.mpl_connect('button_press_event', onclick)

In [22]:
display(old_coord_widget_x, old_coord_widget_y)

FloatText(value=0.0)

FloatText(value=0.0)

In [23]:
display(old_coord_widget_x, old_coord_widget_y)
display(new_coord_widget_x, new_coord_widget_y)

FloatText(value=0.0)

FloatText(value=0.0)

# old

In [125]:
coords

[(79.52885298295453, 98.8506467084348), (86.02235947646102, 89.32683718462528)]

In [105]:
window_name = 'fish1_xz'
cv2.namedWindow(window_name)
cv2.moveWindow(window_name, 0, 0)


In [ ]:
            cv2.namedWindow(self.window_name)
            cv2.moveWindow(self.window_name, 0, 0)
            self._update_canvas_size()
            self._update_text_locs()
            self._update_canvas()

In [ ]:
# make a widget structure to hold frame results

imCoord = widgets.IntText()

number_layout = widgets.Layout(width='60px')
text_layout = widgets.Layout(width='120px')


In [ ]:
numC

In [88]:
numCams*numFish*numBodyPoints*2

36

In [79]:
fishIdx = 0
bpIdx = 0
f1h_ims = tracks_imCoords[:, global_index_slider.value, fishIdx, bpIdx]
f1h_ims

array([[963.58823176, 841.94716902],
       [351.9815006 , 630.41492003],
       [493.9735449 , 831.18185379]])

In [67]:
widget_list = [ [] for _ in range(3)]

for camIdx in range(3):
    for imCoord_comp_idx in range(2):
        val = f1h_ims[camIdx, imCoord_comp_idx]
        widget_list[camIdx][imCoord_comp_idx].append(widgets.IntText(value=val, layout=number_layout))

IndexError: list index out of range

In [65]:
widget_list

[[], [], []]

In [62]:

number_layout = widgets.Layout(width='60px')
text_layout = widgets.Layout(width='120px')

camNames = ['xz', 'xy', 'yz']
camName = camNames[camIdx]
title = widgets.Text('Fish{0}, {1}'.format(fishIdx, camName), layout=text_layout)




fish1_head_cam_imcoords = widgets.HBox([widgets.IntText(value=7, layout=number_layout),
                                        widgets.IntText(value=7, layout=number_layout)]
                                       
fish1_head_imcoords = widgets.VBox([title] + [fish1_head_cam_imcoords for _ in range(3)], description='test')
#ccc = widgets.IntText(value=7, description='Any:')


fish1_head_imcoords


In [87]:
fish1_head_imcoords[0]

TypeError: 'VBox' object does not support indexing

# old

In [14]:
def onclick(event):
    global current_timer
    j = 0
    cv = event.canvas
    fig = cv.figure
    frames = 100

    def update():
        nonlocal j
        # restore the background
        cv.restore_region(bg_cache)

        # update the data

        ii = j * np.pi / frames
        y1 = y * np.sin(ii)

        # update the line on the fixed limit axes
        line1.set_ydata(y1)
        ax.draw_artist(line1)

        # update the line on the limit adjusted axes
        line2.set_ydata(-y1)
        ax2.set_ylim(y1.min(), y1.max())
        fig.draw_artist(ax2)

        # update the screen
        cv.blit(fig.bbox)
        # update counter
        j += 1
        # if we are more than 100 times through, bail!
        if j > frames:
            print(f'fps : {frames / (time.time() - start_time)}')
            return False

    start_time = time.time()
    current_timer = cv.new_timer(
        interval=100, callbacks=[(update, (), {})])

    current_timer.start()


def ondraw(event):
    global bg_cache
    cv = event.canvas
    fig = cv.figure
    bg_cache = cv.copy_from_bbox(fig.bbox)

    ax.draw_artist(line1)
    fig.draw_artist(ax2)


x = np.linspace(0, 2 * np.pi, 100)
y = np.sin(x)

plt.ion()
fig = plt.figure()
ax = fig.add_subplot(1, 2, 1)
line1, = ax.plot(x, y, animated=True)

ax2 = fig.add_subplot(1, 2, 2)
ax2.set_animated(True)
line2, = ax2.plot(x, y)


d_cid = fig.canvas.mpl_connect('draw_event', ondraw)
bp_cid = fig.canvas.mpl_connect('button_press_event', onclick)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:


plt.ioff()


# set the slider we are using
slider = global_index_slider

fig = plt.figure()
fig.canvas.header_visible = True
fig.canvas.layout.min_height = '400px'
plt.title('Plotting: global_fIdx = {0})'.format(slider.value))
plt.ylim(0, 40)
#plt.ticklabel_format(useOffset=False, style='plain')

fishIdx = 0
bpIdx = 0
dim = 0
plot_frame_width = 50

# get the complete data
data_all_time = np.copy(tracks_3D[:, fishIdx, bpIdx, dim])

# make the intial data
if slider.value < plot_frame_width:
    print('early')
    data = data_all_time[:2*plot_frame_width]
    timearr = np.arange(2*plot_frame_width)
elif totalnumFrames - slider.value < plot_frame_width:
    print('middle')
    data = data_all_time[-2*plot_frame_width:]
    timearr = np.arange(totalnumFrames-2*plot_frame_width, totalnumFrames, 1)
else:
    print('late')
    data = data_all_time[slider.value-plot_frame_width:slider.value+plot_frame_width]
    timearr = np.arange(slider.value-plot_frame_width, slider.value+plot_frame_width, 1)


lines = plt.plot(timearr, data)

def update_lines(change):
    plt.title('Plotting: global_fIdx = {0})'.format(change.new))
    if change.new < plot_frame_width:
        data = data_all_time[:2*plot_frame_width]
        timearr = np.arange(2*plot_frame_width)
    elif totalnumFrames - change.new < plot_frame_width:
        data = data_all_time[-2*plot_frame_width:]
        timearr = np.arange(totalnumFrames-2*plot_frame_width, totalnumFrames, 1)
    else:
        data = data_all_time[change.new-plot_frame_width:change.new+plot_frame_width]
        timearr = np.arange(change.new-plot_frame_width, change.new+plot_frame_width, 1)
    lines[0].set_data(timearr, data)
    plt.xlim(timearr[0], timearr[-1])
    fig.canvas.draw_idle()
    fig.canvas.flush_events()
    

slider.observe(update_lines, names='value')

aaa= AppLayout(
    center=fig.canvas,
    footer=slider,
    pane_heights=[0, 6, 1]
)

aaa

late


AppLayout(children=(IntSlider(value=65293, description='Frame', layout=Layout(grid_area='footer', height='50px…

In [ ]:
def plot_trajectory_data():
    ''' Make a plot of the trajectory information
    '''
    return


def update_trajectory_plot_on_global_frame_change():
    ''' Update the trajectory data plot
    '''

# Test with interactive plots

In [14]:
from math import pi

import numpy as np

import branca

from ipywidgets import VBox, IntSlider

from ipycanvas import Canvas, MultiCanvas, hold_canvas

In [15]:
class Plot(MultiCanvas):
    def __init__(self, x, y, color=None, scheme=branca.colormap.linear.RdBu_11):
        super(Plot, self).__init__(3, width=800, height=600, sync_image_data=True)

        self.color = color
        self.scheme = scheme
        
        self.background_color = '#f7f7f7'

        self.init_plot(x, y)

    def init_plot(self, x, y, color=None, scheme=None):
        self.x = x
        self.y = y
        self.color = color if color is not None else self.color
        self.scheme = scheme if scheme is not None else self.scheme

        padding = 0.1
        padding_x = padding * self.size[0]
        padding_y = padding * self.size[1]

        # TODO Fix drawarea max: It should be (canvas.size - padding)
        self.drawarea = (drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y) = (padding_x, padding_y, self.size[0] - 2 * padding_x, self.size[1] - 2 * padding_y)

        min_x, min_y, max_x, max_y = np.min(x), np.min(y), np.max(x), np.max(y)

        dx = max_x - min_x
        dy = max_y - min_y

        # Turns a data coordinate into pixel coordinate
        self.scale_x = lambda x: drawarea_max_x * (x - min_x) / dx + drawarea_min_x
        self.scale_y = lambda y: drawarea_max_y * (1 - (y - min_y) / dy) + drawarea_min_y

        # Turns a pixel coordinate into data coordinate
        self.unscale_x = lambda sx: (sx - drawarea_min_x) * dx / drawarea_max_x + min_x
        self.unscale_y = lambda sy: (1 - ((sy - drawarea_min_y) / drawarea_max_y)) * dy + min_y

        self.colormap = None
        if self.color is not None:
            self.colormap = self.scheme.scale(np.min(self.color), np.max(self.color))

    def draw_background(self):
        drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y = self.drawarea

        background = self[0]

        # Draw background
        background.fill_style = self.background_color
        background.global_alpha = 0.3
        background.fill_rect(drawarea_min_x, drawarea_min_y, drawarea_max_x, drawarea_max_y)
        background.global_alpha = 1

        # Draw grid and ticks
        n_lines = 10
        background.fill_style = 'black'
        background.stroke_style = '#8c8c8c'
        background.line_width = 1
        background.begin_path()

        for i in range(n_lines):
            j = i / (n_lines - 1)
            line_x = drawarea_max_x * j + drawarea_min_x
            line_y = drawarea_max_y * j + drawarea_min_y

            # Line on the y axis
            background.move_to(line_x, drawarea_min_y)
            background.line_to(line_x, drawarea_max_y + drawarea_min_y)

            # Line on the x axis
            background.move_to(drawarea_min_x, line_y)
            background.line_to(drawarea_max_x + drawarea_min_x, line_y)

            # Draw y tick
            background.text_align = 'right'
            background.text_baseline = 'middle'
            background.fill_text('{0:.2e}'.format(self.unscale_y(line_y)), drawarea_min_x * 0.95, line_y)

            # Draw x tick
            background.text_align = 'center'
            background.text_baseline = 'top'
            background.fill_text('{0:.2e}'.format(self.unscale_x(line_x)), line_x, drawarea_max_y + drawarea_min_y + drawarea_min_y * 0.05)

        background.stroke()
        background.close_path()

In [16]:
class ScatterPlot(Plot):
    def __init__(self, x, y, size, color, scheme=branca.colormap.linear.RdBu_11, stroke_color='black'):
        super(ScatterPlot, self).__init__(x, y, color, scheme)

        self.dragging = False
        self.sizes = size
        self.stroke_color = stroke_color

        self.n_marks = min(x.shape[0], y.shape[0], size.shape[0], color.shape[0])

        # Index of the dragged point
        self.i_mark = -1

        self[2].on_mouse_down(self.mouse_down_handler)
        self[2].on_mouse_move(self.mouse_move_handler)
        self[2].on_mouse_up(self.mouse_up_handler)

        self.draw()

    def draw(self):
        with hold_canvas(self):
            self.clear()
            plot_layer = self[1]

            plot_layer.save()

            self.draw_background()

            # Draw scatter
            plot_layer.stroke_style = self.stroke_color

            for idx in range(self.n_marks):
                plot_layer.fill_style = self.colormap(self.color[idx])

                mark_x = self.scale_x(self.x[idx])
                mark_y = self.scale_y(self.y[idx])
                mark_size = self.sizes[idx]

                plot_layer.fill_arc(mark_x, mark_y, mark_size, 0, 2 * pi)
                plot_layer.stroke_arc(mark_x, mark_y, mark_size, 0, 2 * pi)

            plot_layer.restore()

    def mouse_down_handler(self, pixel_x, pixel_y):
        plot_layer = self[1]

        for idx in range(self.n_marks):
            mark_x = self.x[idx]
            mark_y = self.y[idx]
            mark_size = self.sizes[idx]

            if (pixel_x > self.scale_x(mark_x) - mark_size and pixel_x < self.scale_x(mark_x) + mark_size and
                pixel_y > self.scale_y(mark_y) - mark_size and pixel_y < self.scale_y(mark_y) + mark_size):
                self.i_mark = idx
                self.dragging = True

                with hold_canvas(plot_layer):
                    plot_layer.fill_style = self.background_color
                    plot_layer.stroke_style = self.colormap(self.color[self.i_mark])

                    plot_layer.fill_arc(self.scale_x(mark_x), self.scale_y(mark_y), mark_size, 0, 2 * pi)
                    plot_layer.stroke_arc(self.scale_x(mark_x), self.scale_y(mark_y), mark_size, 0, 2 * pi)
                break

    def mouse_move_handler(self, pixel_x, pixel_y):
        if self.dragging and self.i_mark != -1:
            interaction_layer = self[2]

            unscaled_x = self.unscale_x(pixel_x)
            unscaled_y = self.unscale_y(pixel_y)

            with hold_canvas(interaction_layer):
                interaction_layer.clear()
                interaction_layer.fill_style = self.colormap(self.color[self.i_mark])
                interaction_layer.stroke_style = self.stroke_color

                self.x[self.i_mark] = unscaled_x
                self.y[self.i_mark] = unscaled_y

                interaction_layer.fill_arc(pixel_x, pixel_y, self.sizes[self.i_mark], 0, 2 * pi)
                interaction_layer.stroke_arc(pixel_x, pixel_y, self.sizes[self.i_mark], 0, 2 * pi)

    def mouse_up_handler(self, pixel_x, pixel_y):
        self.dragging = False

        self.draw()

        interaction_layer = self[2]
        interaction_layer.clear()

In [17]:
class LinePlot(Plot):
    def __init__(self, x, y, line_color='#749cb8', line_width=2):
        super(LinePlot, self).__init__(x, y)

        self.line_color = line_color
        self.line_width = line_width

        self.draw()

    def update(self, x, y, line_color=None, line_width=None):
        self.init_plot(x, y)

        self.line_color = line_color if line_color is not None else self.line_color
        self.line_width = line_width if line_width is not None else self.line_width

        self.draw()

    def draw(self):
        with hold_canvas(self):
            self.clear()
            plot_layer = self[1]
            plot_layer.save()

            self.draw_background()

            # Draw lines
            n_points = min(self.x.shape[0], self.y.shape[0])

            plot_layer.begin_path()
            plot_layer.stroke_style = self.line_color
            plot_layer.line_width = self.line_width
            plot_layer.line_join = 'bevel'
            plot_layer.line_cap = 'round'
            plot_layer.move_to(self.scale_x(self.x[0]), self.scale_y(self.y[0]))
            for idx in range(1, n_points):
                plot_layer.line_to(
                    self.scale_x(self.x[idx]), self.scale_y(self.y[idx])
                )

            plot_layer.stroke()
            plot_layer.close_path()

            plot_layer.restore()

In [18]:
n_points = 1_000

In [19]:
x = np.random.rand(n_points)
y = np.random.rand(n_points)
sizes = np.random.randint(2, 8, n_points)
colors = np.random.rand(n_points) * 10 - 2

plot = ScatterPlot(x, y, sizes, colors, branca.colormap.linear.viridis, stroke_color='white')
plot

ScatterPlot(height=600, sync_image_data=True, width=800)

In [1]:
## ----  Make a plotting widget which is tied to "global fIdx" ----  ##
t = widgets.IntText(value=60)

## WORK HERE ON MAKING PLOT INFO FUNCTION

if t.value < 50:
    x_vals = np.arange(0, 100)
    f1hx = tracks_3D_test[:100, 0, 0, 0]
    f1tx = tracks_3D_test[:100, 0, 2, 0]
    f2hx = tracks_3D_test[:100, 1, 0, 0]
    f2tx = tracks_3D_test[:100, 1, 2, 0]
    #plt.xlim(0,100)
    #plt.xticks(np.arange(0,100,10))

elif totalnumFrames - t.value < 50:
    x_vals = np.arange(totalnumFrames-50, totalnumFrames)
    f1hx = tracks_3D_test[-100:, 0, 0, 0]
    f1tx = tracks_3D_test[-100:, 0, 2, 0]
    f2hx = tracks_3D_test[-100:, 1, 0, 0]
    f2tx = tracks_3D_test[-100:, 1, 2, 0] 
    #plt.xlim(totLalnumFrames-100, totalnumFrames)
    #plt.xticks(np.arange(totalnumFrames-100,totalnumFrames,10))

else:
    x_vals = np.arange(t.value-50, t.value+50)
    f1hx = tracks_3D_test[t.value-50:t.value+50, 0, 0, 0]
    f1tx = tracks_3D_test[t.value-50:t.value+50, 0, 2, 0]
    f2hx = tracks_3D_test[t.value-50:t.value+50, 1, 0, 0]
    f2tx = tracks_3D_test[t.value-50:t.value+50, 1, 2, 0]


fig_plot = plt.figure(title='f1hx')

p1 = plt.plot(x_vals, f1hx)

# p1 = plt.plot(x_vals, f1hx, label='f1hx')
# p2 = plt.plot(x_vals, f2hx, label='f2hx')
# p3 = plt.plot(x_vals, f1tx, label='f1tx')
# p4 = plt.plot(x_vals, f2tx, label='f2tx')

widgets.link((t, 'value'), (global_index.value))
    
fig_plot   

NameError: name 'widgets' is not defined

In [97]:
t.value

60

In [55]:
display(plot_widget)

interactive(children=(IntSlider(value=0, description='Frame', layout=Layout(height='50px', width='90%'), max=5…

In [22]:
## ---- Interpolate the heads for making movies without gaps ----- ##

def fill_nan(A):
    '''
    Thanks - https://stackoverflow.com/a/9815522
    interpolate to fill nan values
    '''
    inds = np.arange(A.shape[0])
    good = np.where(np.isfinite(A))
    f = interpolate.interp1d(inds[good], A[good],bounds_error=False)
    B = np.where(np.isfinite(A),A,f(inds))
    return B


# interpolate the fish_pecs so we dont blink

xz_heads = np.copy(original_instances_new[0,:,:,0,:])
xy_heads = np.copy(original_instances_new[1,:,:,0,:])
yz_heads = np.copy(original_instances_new[2,:,:,0,:])

xz_heads_interpd = np.copy(xz_heads)
xy_heads_interpd = np.copy(xy_heads)
yz_heads_interpd = np.copy(yz_heads)

numFish=2
for fishIdx in range(numFish):
    
    xz_heads_x = np.copy(xz_heads[:, fishIdx, 0])
    xz_heads_y = np.copy(xz_heads[:, fishIdx, 1])
    xz_heads_x_interpd = fill_nan(xz_heads_x)
    xz_heads_y_interpd = fill_nan(xz_heads_y)
    xz_heads_interpd[:, fishIdx, 0] = xz_heads_x_interpd
    xz_heads_interpd[:, fishIdx, 1] = xz_heads_y_interpd
    
    xy_heads_x = np.copy(xy_heads[:, fishIdx, 0])
    xy_heads_y = np.copy(xy_heads[:, fishIdx, 1])
    xy_heads_x_interpd = fill_nan(xy_heads_x)
    xy_heads_y_interpd = fill_nan(xy_heads_y)
    xy_heads_interpd[:, fishIdx, 0] = xy_heads_x_interpd
    xy_heads_interpd[:, fishIdx, 1] = xy_heads_y_interpd
    
    yz_heads_x = np.copy(yz_heads[:, fishIdx, 0])
    yz_heads_y = np.copy(yz_heads[:, fishIdx, 1])
    yz_heads_x_interpd = fill_nan(yz_heads_x)
    yz_heads_y_interpd = fill_nan(yz_heads_y)
    yz_heads_interpd[:, fishIdx, 0] = yz_heads_x_interpd
    yz_heads_interpd[:, fishIdx, 1] = yz_heads_y_interpd
    
    
# make a list of crop_centers for the frames
crop_centers = [xz_heads_interpd, xy_heads_interpd, yz_heads_interpd]
    
    

In [23]:
def _pad_frame(frame, padding_depth):
    ''' Return a padded version of the frame '''
    # grab the padding from the image
    padding = frame[0:padding_depth,:]
    # make the padded frame
    frame = np.vstack((padding, frame, padding))
    return frame

padding_depth = 70
cropSize = (80,80)
bw = 20

In [9]:

def plot_crops(t=0):
    plt.figure()
    
    edges = [[0,1], [1,2]]
    
    # for the current splitdata_idx, get the movies
    video_paths = [xzMovPath, xyMovPath, yzMovPath]

    # open a cap for these movies
    caps = [cv2.VideoCapture(video_path.value) for video_path in video_paths]

    # make a list of colors
    no_id_color = (255,255,255) # white

    # jumpt to the current frame
    [cap.set(1,local_index.value) for cap in caps]

    # grab the frame from each camera
    if caps[0].isOpened() and caps[1].isOpened() and caps[2].isOpened():  
        # grab the 3 images
        ret_xz, frame_xz = caps[0].read()
        if not ret_xz:
            raise TypeError('xz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
        ret_xy, frame_xy = caps[1].read()
        if not ret_xy:
            raise TypeError('xy movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
        ret_yz, frame_yz = caps[2].read()
        if not ret_yz:
            raise TypeError('yz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
        frames = [frame_xz, frame_xy, frame_yz]

    # draw the info
    for camIdx in range(3):
        frame = frames[camIdx]
        frame_cam_tracks = original_instances_new[camIdx, t]
        for fishIdx in range(numFish):
            # draw the bodypoints
            for bpIdx in range(numBodyPoints):
                # get the data
                imCoord = frame_cam_tracks[fishIdx, bpIdx]
                #get the color
                color = ( int (no_id_color[ 0 ]), int(no_id_color[ 1 ]), int(no_id_color[ 2 ]))
                if np.all(np.isnan(imCoord)):
                    continue
                cv2.circle(frame, (int(imCoord[0]), int(imCoord[1])), radius=2, color=color, thickness=-1)
            # draw the lines
            for edge in edges:
                pt1_bpIdx = edge[0]
                pt2_bpIdx = edge[1]
                pt1_imCoord = frame_cam_tracks[fishIdx, pt1_bpIdx]
                pt2_imCoord = frame_cam_tracks[fishIdx, pt2_bpIdx]
                #print(pt1_imCoord)
                # parse the points for format and emptiness
                if np.all(np.isnan(pt1_imCoord)):
                    pt1 =tuple([np.NaN, np.NaN])
                else:
                    pt1 = tuple([int(x) for x in pt1_imCoord])
                if np.all(np.isnan(pt2_imCoord)):
                    pt2 =tuple([np.NaN, np.NaN])
                else:
                    pt2 = tuple([int(x) for x in pt2_imCoord])
                if ~np.all(np.isnan(pt1)) and ~np.all(np.isnan(pt2)):
                    cv2.line(frame, pt1=pt1,  pt2=pt2, color=color, thickness=1, lineType=cv2.LINE_AA)
            
            

    # take crops around fish1 and fish2
    crops_fish1 = []
    crops_fish2 = []
    for camIdx in range(3):
        fish1_head = crop_centers[camIdx][t, 0]
        fish2_head = crop_centers[camIdx][t, 1]
        # get the center of the crop, adjusted for padding
        cropCenter1 = [fish1_head[0], fish1_head[1]+padding_depth]
        cropCenter2 = [fish2_head[0], fish2_head[1]+padding_depth]

        if np.all(np.isnan(fish1_head)):
            crop = np.zeros((cropSize[0]*2, cropSize[1]*2, 3), dtype=np.uint8)
            crops_fish1.append(crop)
        else:
            frame = frames[camIdx]
            frame = _pad_frame(frame, padding_depth)
            crop = frame[int(cropCenter1[1]-cropSize[1]):int(cropCenter1[1]+cropSize[1]),
                         int(cropCenter1[0]-cropSize[0]):int(cropCenter1[0]+cropSize[0])]
            crops_fish1.append(crop)
            
        if np.all(np.isnan(fish2_head)):
            crop = np.zeros((cropSize[0]*2, cropSize[1]*2, 3), dtype=np.uint8)
            crops_fish2.append(crop)
        else:
            frame = frames[camIdx]
            frame = _pad_frame(frame, padding_depth)
            crop = frame[int(cropCenter2[1]-cropSize[1]):int(cropCenter2[1]+cropSize[1]),
                         int(cropCenter2[0]-cropSize[0]):int(cropCenter2[0]+cropSize[0])]
            crops_fish2.append(crop)

    # combine the crops
    barrier = np.zeros((cropSize[0]*2, bw, 3), dtype=np.uint8)
    movFrame1 = np.hstack([crops_fish1[0], barrier, crops_fish1[1], barrier, crops_fish1[2]])
    movFrame2 = np.hstack([crops_fish2[0], barrier, crops_fish2[1], barrier, crops_fish2[2]])

    # close everything before we finish
    [caps[capIdx].release() for capIdx in range(3)];
    
    plt.figure(figsize=(8,4))
    plt.imshow(movFrame1)
    plt.figure(figsize=(8,4))
    plt.imshow(movFrame2)
    return


crop_widget = interactive(plot_crops, t=global_index_slider, 
                          layout=Layout(width='100%'))

In [25]:
def plot_tracked_crops(t=0):
    plt.figure()
    
    edges = [[0,1], [1,2]]
    
    # for the current splitdata_idx, get the movies
    video_paths = [xzMovPath, xyMovPath, yzMovPath]

    # open a cap for these movies
    caps = [cv2.VideoCapture(video_path.value) for video_path in video_paths]

    # make a list of colors
    colors = [(255,0,0), (0,0,255)] # white

    # jumpt to the current frame
    [cap.set(1,local_index.value) for cap in caps]

    # grab the frame from each camera
    if caps[0].isOpened() and caps[1].isOpened() and caps[2].isOpened():  
        # grab the 3 images
        ret_xz, frame_xz = caps[0].read()
        if not ret_xz:
            raise TypeError('xz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
        ret_xy, frame_xy = caps[1].read()
        if not ret_xy:
            raise TypeError('xy movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
        ret_yz, frame_yz = caps[2].read()
        if not ret_yz:
            raise TypeError('yz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
        frames = [frame_xz, frame_xy, frame_yz]

    # draw the info
    for camIdx in range(3):
        frame = frames[camIdx]
        frame_cam_tracks = tracks_imCoords[camIdx, t]
        for fishIdx in range(numFish):
            # draw the bodypoints
            for bpIdx in range(numBodyPoints):
                # get the data
                imCoord = frame_cam_tracks[fishIdx, bpIdx]
                #get the color
                color = colors[fishIdx]
                color = ( int (color[ 0 ]), int(color[ 1 ]), int(color[ 2 ]))
                if np.all(np.isnan(imCoord)):
                    continue
                cv2.circle(frame, (int(imCoord[0]), int(imCoord[1])), radius=2, color=color, thickness=-1)
            # draw the lines
            for edge in edges:
                pt1_bpIdx = edge[0]
                pt2_bpIdx = edge[1]
                pt1_imCoord = frame_cam_tracks[fishIdx, pt1_bpIdx]
                pt2_imCoord = frame_cam_tracks[fishIdx, pt2_bpIdx]
                #print(pt1_imCoord)
                # parse the points for format and emptiness
                if np.all(np.isnan(pt1_imCoord)):
                    pt1 =tuple([np.NaN, np.NaN])
                else:
                    pt1 = tuple([int(x) for x in pt1_imCoord])
                if np.all(np.isnan(pt2_imCoord)):
                    pt2 =tuple([np.NaN, np.NaN])
                else:
                    pt2 = tuple([int(x) for x in pt2_imCoord])
                if ~np.all(np.isnan(pt1)) and ~np.all(np.isnan(pt2)):
                    cv2.line(frame, pt1=pt1,  pt2=pt2, color=color, thickness=1, lineType=cv2.LINE_AA)
            
            

    # take crops around fish1 and fish2
    crops_fish1 = []
    crops_fish2 = []
    for camIdx in range(3):
        fish1_head = crop_centers[camIdx][t, 0]
        fish2_head = crop_centers[camIdx][t, 1]
        # get the center of the crop, adjusted for padding
        cropCenter1 = [fish1_head[0], fish1_head[1]+padding_depth]
        cropCenter2 = [fish2_head[0], fish2_head[1]+padding_depth]

        if np.all(np.isnan(fish1_head)):
            crop = np.zeros((cropSize[0]*2, cropSize[1]*2, 3), dtype=np.uint8)
            crops_fish1.append(crop)
        else:
            frame = frames[camIdx]
            frame = _pad_frame(frame, padding_depth)
            crop = frame[int(cropCenter1[1]-cropSize[1]):int(cropCenter1[1]+cropSize[1]),
                         int(cropCenter1[0]-cropSize[0]):int(cropCenter1[0]+cropSize[0])]
            crops_fish1.append(crop)
            
        if np.all(np.isnan(fish2_head)):
            crop = np.zeros((cropSize[0]*2, cropSize[1]*2, 3), dtype=np.uint8)
            crops_fish2.append(crop)
        else:
            frame = frames[camIdx]
            frame = _pad_frame(frame, padding_depth)
            crop = frame[int(cropCenter2[1]-cropSize[1]):int(cropCenter2[1]+cropSize[1]),
                         int(cropCenter2[0]-cropSize[0]):int(cropCenter2[0]+cropSize[0])]
            crops_fish2.append(crop)

    # combine the crops
    barrier = np.zeros((cropSize[0]*2, bw, 3), dtype=np.uint8)
    movFrame1 = np.hstack([crops_fish1[0], barrier, crops_fish1[1], barrier, crops_fish1[2]])
    movFrame2 = np.hstack([crops_fish2[0], barrier, crops_fish2[1], barrier, crops_fish2[2]])

    # close everything before we finish
    [caps[capIdx].release() for capIdx in range(3)];
    
    plt.figure(figsize=(8,4))
    plt.imshow(movFrame1)
    plt.figure(figsize=(8,4))
    plt.imshow(movFrame2)
    return


crop_tracked_widget = interactive(plot_tracked_crops, t=global_index_slider, 
                                  layout=Layout(width='100%'))

In [26]:
# -- A tab for choosing a fish -- #

# tab_contents = ['fish_{0}'.format(i+1) for i in range(numFish)]
# children = [widgets.Text(description=name) for name in tab_contents]
# tab = widgets.Tab()
# tab.children = children
# for ii in range(len(children)):
#     tab.set_title(ii, f"tab_{ii}")
# tab


In [27]:
# ---- make the final vertical box for sidecar and display --- ##

splitdata_details_box = VBox([splitdata_chooser, 
                              splitdata_idx, 
                              global_index, 
                              local_index, 
                              global_index_slider])



sc = Sidecar(title='Sidecar Output')
with sc:
    display(splitdata_details_box)

In [46]:
# --- Display the plot and figures --- #

#display(plot_widget, crop_widget, crop_tracked_widget)
display(plot_widget)



interactive(children=(IntSlider(value=0, description='Frame', layout=Layout(height='50px', width='90%'), max=5…

In [ ]:
plt.imshow()

# Bokeh test

In [100]:
from bqplot import pyplot as plt


x = np.arange(0, 100)
y = np.sin(x*2)
z = (np.sin(x*2))**2 + 1
fig_hist = plt.figure(title='test')
hist = plt.plot(x, y);
fig_hist

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top':…

In [102]:
new_y = np.ones((100,))

In [103]:
hist.y = new_y

In [101]:
z.shape

(100,)

In [81]:
from bqplot import pyplot as plt


x = np.arange(0, 100)
y = np.sin(x*2)

fig_hist = plt.figure(title='test')
hist = plt.hist(y);
fig_hist




Figure(axes=[Axis(orientation='vertical', scale=LinearScale()), Axis(scale=LinearScale())], fig_margin={'top':…

In [82]:
hist.bins=80

In [85]:
hist.count

array([ 8.,  0.,  3.,  3.,  2.,  0.,  0.,  1.,  2.,  3.,  2.,  2.,  0.,
        0.,  0.,  1.,  2.,  1.,  2.,  2.,  2.,  0.,  0.,  0.,  0.,  1.,
        1.,  2.,  2.,  1.,  2.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  2.,
        2.,  1.,  1.,  2.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  2.,  1.,
        1.,  2.,  0.,  0.,  0.,  0.,  0.,  2.,  1.,  1.,  2.,  2.,  0.,
        0.,  0.,  0.,  2.,  2.,  1.,  3.,  0.,  0.,  0.,  3.,  4.,  3.,
        0., 10.])

In [36]:
n = 2000
x = np.linspace(0, 10, n)

np.random.seed(0)

y= np.cumsum(np.random.randn(n)*10).astype(int)


fig_hist = plt.figure(title="Histogram test")
hist = plt.hist(y, bins=25)
fig_hist

Figure(axes=[Axis(orientation='vertical', scale=LinearScale()), Axis(scale=LinearScale())], fig_margin={'top':…

In [51]:
hist.bins = 100

In [108]:


from bokeh.plotting import figure, output_file, show, output_notebook

# prepare some data
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

# output to static HTML file
#output_file("lines.html")
output_notebook()

# create a new plot with a title and axis labels
p = figure(title="simple line example", x_axis_label='x', y_axis_label='y')

# add a line renderer with legend and line thickness
p.line(x, y, legend_label="Temp.", line_width=2)

# show the results
show(p)



Loading BokehJS ...

In [88]:
from bokeh.models import TapTool


In [82]:
t = 100

In [5]:
# create a new plot with a title and axis labels
p = figure(title="Some X data", x_axis_label='frames', y_axis_label='(cm)')
p.add_tools(TapTool())


# prepare some data
x_vals = np.arange(t-50, t+50)

if t < 50:
    x_vals = np.arange(0, 100)
    f1hx = tracks_3D_test[:100, 0, 0, 0]
    f1tx = tracks_3D_test[:100, 0, 2, 0]
    f2hx = tracks_3D_test[:100, 1, 0, 0]
    f2tx = tracks_3D_test[:100, 1, 2, 0]
    #ax.set_xlim(0,100)
    #ax.set_xticks(np.arange(0,100,10))

elif totalnumFrames - t < 50:
    x_vals = np.arange(totalnumFrames-50, totalnumFrames)
    f1hx = tracks_3D_test[-100:, 0, 0, 0]
    f1tx = tracks_3D_test[-100:, 0, 2, 0]
    f2hx = tracks_3D_test[-100:, 1, 0, 0]
    f2tx = tracks_3D_test[-100:, 1, 2, 0] 
    #ax.set_xlim(totalnumFrames-100, totalnumFrames)
    #ax.set_xticks(np.arange(totalnumFrames-100,totalnumFrames,10))

else:
    x_vals = np.arange(t-50, t+50)
    f1hx = tracks_3D_test[t-50:t+50, 0, 0, 0]
    f1tx = tracks_3D_test[t-50:t+50, 0, 2, 0]
    f2hx = tracks_3D_test[t-50:t+50, 1, 0, 0]
    f2tx = tracks_3D_test[t-50:t+50, 1, 2, 0]
    #ax.set_xlim(t-50,t+50)
    #ax.set_xticks(np.arange(t-50,t+50,10))
    
    
    
# add a line renderer with legend and line thickness
p.line(x_vals, f1hx, legend_label="Temp.", line_width=2)

# show the results
show(p)

NameError: name 'TapTool' is not defined

In [93]:
p.selected.indices

AttributeError: 'Figure' object has no attribute 'selected'

In [ ]:
def plot_ts(t=0):
    fig = plt.figure(figsize=(12,6))
    ax = fig.add_subplot(1,1,1)
    
    #x_vals = np.arange(t-50, t+50)
    
    if t < 50:
        x_vals = np.arange(0, 100)
        f1hx = tracks_3D_test[:100, 0, 0, 0]
        f1tx = tracks_3D_test[:100, 0, 2, 0]
        f2hx = tracks_3D_test[:100, 1, 0, 0]
        f2tx = tracks_3D_test[:100, 1, 2, 0]
        ax.set_xlim(0,100)
        ax.set_xticks(np.arange(0,100,10))
        
    elif totalnumFrames - t < 50:
        x_vals = np.arange(totalnumFrames-50, totalnumFrames)
        f1hx = tracks_3D_test[-100:, 0, 0, 0]
        f1tx = tracks_3D_test[-100:, 0, 2, 0]
        f2hx = tracks_3D_test[-100:, 1, 0, 0]
        f2tx = tracks_3D_test[-100:, 1, 2, 0] 
        ax.set_xlim(totalnumFrames-100, totalnumFrames)
        ax.set_xticks(np.arange(totalnumFrames-100,totalnumFrames,10))
    
    else:
        x_vals = np.arange(t-50, t+50)
        f1hx = tracks_3D_test[t-50:t+50, 0, 0, 0]
        f1tx = tracks_3D_test[t-50:t+50, 0, 2, 0]
        f2hx = tracks_3D_test[t-50:t+50, 1, 0, 0]
        f2tx = tracks_3D_test[t-50:t+50, 1, 2, 0]
        ax.set_xlim(t-50,t+50)
        ax.set_xticks(np.arange(t-50,t+50,10))
        
    ax.vlines(t, 0 , 40)

    ax.plot(x_vals, f1hx, label='f1hx')
    ax.plot(x_vals, f2hx, label='f2hx')
    ax.plot(x_vals, f1tx, label='f1tx')
    ax.plot(x_vals, f2tx, label='f2tx')
    
    max_val = np.nanmax([np.nanmax(f1hx),np.nanmax(f2hx),np.nanmax(f1tx),np.nanmax(f2tx)])
    min_val = np.nanmin([np.nanmin(f1hx),np.nanmin(f2hx),np.nanmin(f1tx),np.nanmin(f2tx)])
    ax.set_ylim(min_val, max_val)
    #ax.set_xticks(np.arange(t-50,t+50,10))
    #ax.set_xlim(t-50,t+50)
    
    ax.set_xlabel('Frames')
    ax.set_ylabel('(cm)')
    ax.ticklabel_format(useOffset=False)
    fig.canvas.draw()
    ax.legend(loc='upper right')
    
plot_widget = interactive(plot_ts, t=global_index_slider)

# Quantifying missing data

## In the raw network output

In [156]:
def count_numFish_found(original_instances):
    ''' Return the number of fish found in each frame for each camera view
    '''
    # set all zeros to NaNs
    original_instances = original_instances.astype('float')
    original_instances[original_instances==0] = np.NaN
    numCams, numFrames, numFish, numBps, _ = original_instances.shape
    numFishFound = np.zeros((numCams, numFrames))
    for camIdx in range(numCams):
        for fIdx in range(numFrames):
            fishFound = 0
            for fishIdx in range(numFish):
                if ~np.all(np.isnan(original_instances[camIdx, fIdx, fishIdx])):
                    fishFound += 1
            numFishFound[camIdx, fIdx] = fishFound
    return numFishFound
                    
            
        
def count_numBodypoints_found(original_instances):
    ''' Return the number of bps found in each frame for each camera view
    '''
    # set all zeros to NaNs
    original_instances = original_instances.astype('float')
    original_instances[original_instances==0] = np.NaN
    numCams, numFrames, numFish, numBps, _ = original_instances.shape
    numBps_arr = np.zeros((numCams, numFrames))
    for camIdx in range(numCams):
        for fIdx in range(numFrames):
            numBps_found = 0
            for fishIdx in range(numFish):
                for bpIdx in range(numBps):
                    if ~np.all(np.isnan(original_instances[camIdx, fIdx, fishIdx, bpIdx])):
                        numBps_found += 1
            #import pdb; pdb.set_trace()
            numBps_arr[camIdx, fIdx] = numBps_found
    return numBps_arr



# get the number of fish and bps found by for each data
t0 = time.time()

# old
numFish_found_old = count_numFish_found(original_instances_old)
numFish_found_mean_old = np.mean(numFish_found_old, axis=1)
numBps_found_old = count_numBodypoints_found(original_instances_old)
numBps_found_mean_old = np.mean(numBps_found_old, axis=1)

# sLEAP
numFish_found_new = count_numFish_found(original_instances_new)
numFish_found_mean_new = np.mean(numFish_found_new, axis=1)
numBps_found_new = count_numBodypoints_found(original_instances_new)
numBps_found_mean_new = np.mean(numBps_found_new, axis=1)

tE = time.time()
print(tE-t0)

126.00515294075012


In [157]:
print('-- Old --')
print(numFish_found_mean_old)
print(numBps_found_mean_old)
print()

print('-- new --')
print(numFish_found_mean_new)
print(numBps_found_mean_new)
print()

-- Old --
[1.5443905  1.93911859 1.3730822 ]
[4.47758809 5.70549246 4.08892245]

-- new --
[1.80652983 1.9634082  1.83032934]
[5.37825809 5.86898911 5.43987066]



## At the trajectory level

In [173]:
# How many frames have full information? 

t0 = time.time()

full_data_test = np.zeros((totalnumFrames,))
full_data_old = np.zeros((totalnumFrames,))

for fIdx in range(totalnumFrames):

    f_data_test = tracks_3D_test[fIdx]
    f_data_old = tracks_3D_old[fIdx]

    if ~np.any(np.isnan(f_data_test)):
        full_data_test[fIdx] = 1
        
    if ~np.any(np.isnan(f_data_old)):
        full_data_old[fIdx] = 1
        
        
counts, vals = np.unique(full_data_old, return_counts=True)
old_full_data_summary = list(zip(counts, vals))
old_full_fraction = old_full_data_summary[1][1] / totalnumFrames

counts, vals = np.unique(full_data_test, return_counts=True)
test_full_data_summary = list(zip(counts, vals))
test_full_fraction = test_full_data_summary[1][1] / totalnumFrames

print('-- Percentage of frames with full information --')
print('Old networks: {0} %'.format(old_full_fraction*100))
print('New networks: {0} %'.format(test_full_fraction*100))
     
        
print()
print('----')
tE = time.time()
print('time: ', tE-t0, 's')


-- Percentage of frames with full information --
Old networks: 68.07705257369861 %
New networks: 85.2467312025469 %

----
time:  5.74419379234314 s


In [172]:
# How many frames have at least some information

t0 = time.time()

some_fish_data_test = np.zeros((totalnumFrames,))
some_fish_data_old = np.zeros((totalnumFrames,))

for fIdx in range(totalnumFrames):

    f_data_test = tracks_3D_test[fIdx]
    f_data_old = tracks_3D_old[fIdx]

    if ~np.all(np.isnan(f_data_test)):
        some_fish_data_test[fIdx] = 1
        
    if ~np.all(np.isnan(f_data_old)):
        some_fish_data_old[fIdx] = 1
        
        
counts, vals = np.unique(some_fish_data_old, return_counts=True)
old_some_fish_data_summary = list(zip(counts, vals))
old_some_fish_fraction = old_some_fish_data_summary[1][1] / totalnumFrames

counts, vals = np.unique(some_fish_data_test, return_counts=True)
test_some_fish_data_summary = list(zip(counts, vals))
test_some_fish_fraction = test_some_fish_data_summary[1][1] / totalnumFrames

print('-- Percentage of frames with some information --')
print('Old networks: {0} %'.format(old_some_fish_fraction*100))
print('New networks: {0} %'.format(test_some_fish_fraction*100))
     
        
print()
print('----')
tE = time.time()
print('time: ', tE-t0, 's')

-- Percentage of frames with some information --
Old networks: 87.33401999828666 %
New networks: 97.5983328784344 %

----
time:  5.816539287567139 s


# Plotting

In [174]:
def plot_ts(w=0):
    fig = plt.figure(figsize=(20,8))
    ax = fig.add_subplot(1,1,1)
    
    fishIdx=0
    bpIdx=2
    f1hx = tracks_3D[0+w:100+w, fishIdx, bpIdx, 0] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 0])
    #f1hy = tracks_3D[0+w:100+w, fishIdx, bpIdx, 1] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 1])
    #f1hz = tracks_3D[0+w:100+w, fishIdx, bpIdx, 2] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 2])
    
    fishIdx=0
    bpIdx=2
    ## USING SAME MEAN FOR SUBTRACTION
    f1hx_old = tracks_3D_old[0+w:100+w, fishIdx, bpIdx, 0] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 0])
    #f1hy = tracks_3D[0+w:100+w, fishIdx, bpIdx, 1] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 1])
    #f1hz = tracks_3D[0+w:100+w, fishIdx, bpIdx, 2] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 2])
    
    
    fishIdx=0
    bpIdx=2
    ## USING SAME MEAN FOR SUBTRACTION
    f1hx_test = tracks_3D_test[0+w:100+w, fishIdx, bpIdx, 0] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 0])
    #f1hy = tracks_3D[0+w:100+w, fishIdx, bpIdx, 1] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 1])
    #f1hz = tracks_3D[0+w:100+w, fishIdx, bpIdx, 2] - np.nanmean(tracks_3D[0+w:100+w, fishIdx, bpIdx, 2])
    
    
    ax.plot(f1hx, label='f1hx_new')
    ax.plot(f1hx_old, label='f1hx_old')
    ax.plot(f1hx_test, label='f1hx_test')
    
#     ax.plot(f1hx, label='f1hx')
#     ax.plot(f1hy, label='f1hy')
#     ax.plot(f1hz, label='f1hz')
    ax.set_ylim(-10,10)
    fig.canvas.draw()
    ax.legend()
    
interact(plot_ts, w=widgets.IntSlider(min=0,max=totalnumFrames,step=1,value=10))

interactive(children=(IntSlider(value=10, description='w', max=501943), Output()), _dom_classes=('widget-inter…

<function __main__.plot_ts(w=0)>

In [176]:
for fIdx in range(207673, 210000):
    print(fIdx)
    print()
    print(tracks_3D_test[fIdx])

207673

[[[nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]]]
207674

[[[nan nan nan]
  [nan nan nan]
  [nan nan nan]]

 [[nan nan nan]
  [nan nan nan]
  [nan nan nan]]]
207675

[[[ 6.73607502 32.69742968 -0.66681904]
  [        nan         nan         nan]
  [        nan         nan         nan]]

 [[        nan         nan         nan]
  [ 8.82970089 31.61764188 -0.46101748]
  [ 9.04766555 30.05893042 -0.46536218]]]
207676

[[[ 6.73578165 32.69789819 -0.66690104]
  [        nan         nan         nan]
  [        nan         nan         nan]]

 [[        nan         nan         nan]
  [ 8.82968532 31.61786046 -0.46090694]
  [ 9.0476341  30.05927135 -0.46529833]]]
207677

[[[ 6.73541922 32.69853393 -0.66721306]
  [        nan         nan         nan]
  [        nan         nan         nan]]

 [[        nan         nan         nan]
  [ 8.82961555 31.61767394 -0.46106263]
  [ 9.04752122 30.05934663 -0.46530315]]]
207678

[[[ 6.73486182 32.69

# Analysis

In [10]:
global_fstart = 80000
global_fstop = global_fstart + 200

In [11]:
# get the local info
splitdata_idx1, local_fstart = splitman.return_splitdata_folder_and_local_idx_for_global_frameIdx(global_fstart, return_splitdataIdx=True)
splitdata_name1 = splitdata_names[splitdata_idx1]
print(splitdata_idx1)
print(splitdata_name1)
print(local_fstart)
print()
splitdata_idx2, local_fstop = splitman.return_splitdata_folder_and_local_idx_for_global_frameIdx(global_fstop, return_splitdataIdx=True)
splitdata_name2 = splitdata_names[splitdata_idx2]
print(splitdata_idx2)
print(splitdata_name2)
print(local_fstop)

13
splitdata0022
2000

13
splitdata0022
2200


In [12]:
# get the movie paths
# idx1 == idx2
paths = splitman.splitdata_paths[splitdata_idx1]
video_paths = [path +'.mp4' for path in paths]
video_paths

['/media/stephens-group/guest_drive/labelling/complete_shortfin_experiment/FishTank20200130_153857/D_xz/splitdata0022.mp4',
 '/media/stephens-group/guest_drive/labelling/complete_shortfin_experiment/FishTank20200130_153857/E_xy/splitdata0022.mp4',
 '/media/stephens-group/guest_drive/labelling/complete_shortfin_experiment/FishTank20200130_153857/F_yz/splitdata0022.mp4']

In [89]:
# gather the data for this section of frames
xz_tracks = np.copy(tracks_imCoords[0, global_fstart:global_fstop])
xy_tracks = np.copy(tracks_imCoords[1, global_fstart:global_fstop])
yz_tracks = np.copy(tracks_imCoords[2, global_fstart:global_fstop])
tra_3D = np.copy(tracks_3D[global_fstart:global_fstop])
numFrames = global_fstop - global_fstart

numFrames

200

In [57]:
def update(w=1):
    fig = plt.figure(figsize=(10,6))
    ax = fig.add_subplot(1,1,1)
    frame = make_3D_plot_of_frame_positions(tra_3D[w])
    ax.imshow(frame)
    fig.canvas.draw()
    
interact(update, w=widgets.IntSlider(min=0,max=numFrames,step=1,value=0))

interactive(children=(IntSlider(value=0, description='w', max=200), Output()), _dom_classes=('widget-interact'…

<function __main__.update(w=1)>

In [68]:
xz_tracks.shape

(200, 2, 3, 2)

In [90]:
# interpolate the centers

from scipy.interpolate import interp1d
import numpy as np
from scipy import interpolate

def fill_nan(A):
    '''
    Thanks - https://stackoverflow.com/a/9815522
    interpolate to fill nan values
    '''
    inds = np.arange(A.shape[0])
    good = np.where(np.isfinite(A))
    f = interpolate.interp1d(inds[good], A[good],bounds_error=False)
    B = np.where(np.isfinite(A),A,f(inds))
    return B

# interpolate the fish_pecs so we dont blink
xz_heads = np.copy(xz_tracks[:,:,0])
xy_heads = np.copy(xy_tracks[:,:,0])
yz_heads = np.copy(yz_tracks[:,:,0])
xz_heads_interpd = np.copy(xz_heads)
xy_heads_interpd = np.copy(xy_heads)
yz_heads_interpd = np.copy(yz_heads)

for fishIdx in range(2):
    xz_heads_x = np.copy(xz_heads[:, fishIdx, 0])
    xz_heads_y = np.copy(xz_heads[:, fishIdx, 1])
    xz_heads_x_interpd = fill_nan(xz_heads_x)
    xz_heads_y_interpd = fill_nan(xz_heads_y)
    xz_heads_interpd[:, fishIdx, 0] = xz_heads_x_interpd
    xz_heads_interpd[:, fishIdx, 1] = xz_heads_y_interpd
    
    xy_heads_x = np.copy(xy_heads[:, fishIdx, 0])
    xy_heads_y = np.copy(xy_heads[:, fishIdx, 1])
    xy_heads_x_interpd = fill_nan(xy_heads_x)
    xy_heads_y_interpd = fill_nan(xy_heads_y)
    xy_heads_interpd[:, fishIdx, 0] = xy_heads_x_interpd
    xy_heads_interpd[:, fishIdx, 1] = xy_heads_y_interpd
    
    yz_heads_x = np.copy(yz_heads[:, fishIdx, 0])
    yz_heads_y = np.copy(yz_heads[:, fishIdx, 1])
    yz_heads_x_interpd = fill_nan(yz_heads_x)
    yz_heads_y_interpd = fill_nan(yz_heads_y)
    yz_heads_interpd[:, fishIdx, 0] = yz_heads_x_interpd
    yz_heads_interpd[:, fishIdx, 1] = yz_heads_y_interpd
    
    
crop_centers = [xz_heads_interpd, xy_heads_interpd, yz_heads_interpd]


In [91]:
movieSavePath = '/home/liam/temp/mov1.mp4'

make_3panel_movie(video_paths, xy_tracks, xz_tracks, yz_tracks, movieSavePath, 
                  crop_centers, cropSize=(80,80), bw=20, padding_depth=70,
                  fstart=local_fstart, fstop=local_fstop)

fstart 2000
fstop 2200
2000
2100


# Making a movie of results

In [75]:
def _pad_frame(frame, padding_depth):
    ''' Return a padded version of the frame '''
    # grab the padding from the image
    padding = frame[0:padding_depth,:]
    # make the padded frame
    frame = np.vstack((padding, frame, padding))
    return frame


In [82]:
# Make a movie

def make_3panel_movie(video_paths, xy_tracks, xz_tracks, yz_tracks, movieSavePath, 
                      crop_centers, cropSize=(80,80), bw=20, padding_depth=70,
                      fstart=0, fstop=6000):
    """ Visualize the sLEAP and idTracker data drawn onto the movie
    
    -- args --
    video_path: the filepath of the splitdata.mp4
    **_tracks: the tracked trajectories from the ** camera
    movieSavePath: the filepath to save the new mp4 to
    crop_centers: these are interpolated timeseries of head position, that are
                  used from cropping when making the movie
    
    """
    print('fstart', fstart)
    print('fstop', fstop)
    
    # set up the video maker
    fps = 100
    fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
    resolution = (480+bw+bw, 160) # cols/rows
    out = cv2.VideoWriter(movieSavePath, fourcc, fps, resolution)
    
    # parse some shapes
    numFrames, numFish, numBodyPoints, _ = xy_tracks.shape
    if numFrames > fstop:
        fstop = numFrames
    
    # make a list of colors
    colors = [(0,255,0), (255,255,255)] # blue and green
    
    # open the caps
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    
    # pack up the tracks
    tracks = [xz_tracks, xy_tracks, yz_tracks]
    
    frame_index = 0
    while caps[0].isOpened() and caps[1].isOpened() and caps[2].isOpened():
            
        # grab the 3 images
        ret_xz, frame_xz = caps[0].read()
        if not ret_xz:
            break
        ret_xy, frame_xy = caps[1].read()
        if not ret_xy:
            break
        ret_yz, frame_yz = caps[2].read()
        if not ret_yz:
            break
        frames = [frame_xz, frame_xy, frame_yz]
        
        if frame_index < fstart:
            frame_index += 1
            continue
    
        # update message
        if np.mod(frame_index, 100) == 0:
            print(frame_index)
        
        # draw the info
        for camIdx in range(3):
            frame = frames[camIdx]
            trks = tracks[camIdx]
            for fishIdx in range(numFish):
                for bpIdx in range(numBodyPoints):
                    # get the data
                    imCoord = trks[frame_index-fstart, fishIdx, bpIdx]
                    #get the color
                    color = colors[fishIdx]
                    color = ( int (color[ 0 ]), int(color[ 1 ]), int(color[ 2 ]))
                    if np.all(np.isnan(imCoord)):
                        continue
                    cv2.circle(frame, (int(imCoord[0]), int(imCoord[1])), radius=2, color=color, thickness=-1)
                    
        # take crops around fish1
        crops = []
        for camIdx in range(3):
            fish1_head = crop_centers[camIdx][frame_index-fstart, 0]
            # get the center of the crop, adjusted for padding
            cropCenter = [fish1_head[0], fish1_head[1]+padding_depth]
            
            if np.all(np.isnan(fish1_head)):
                crop = np.zeros((cropSize[0]*2, cropSize[1]*2, 3), dtype=np.uint8)
                crops.append(crop)
            else:
                frame = frames[camIdx]
                frame = _pad_frame(frame, padding_depth)
                crop = frame[int(cropCenter[1]-cropSize[1]):int(cropCenter[1]+cropSize[1]),
                             int(cropCenter[0]-cropSize[0]):int(cropCenter[0]+cropSize[0])]
                crops.append(crop)
            
        # combine the crops
        barrier = np.zeros((cropSize[0]*2, bw, 3), dtype=np.uint8)
        movFrame = np.hstack([crops[0], barrier, crops[1], barrier, crops[2]])
        #movFrame = cv2.resize(movFrame, resolution, interpolation = cv2.INTER_AREA)
        
        # write the frame number
        x1, y1 = 10,10
        label = str(frame_index).zfill(6)
        cv2.putText(movFrame,label,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,0),1)
        # write the frame
        out.write(movFrame)
        frame_index += 1
        if frame_index >= fstop:
            break
        
    # close everything before we finish
    for capIdx in range(3):
        caps[capIdx].release()
    out.release() 
    return

In [53]:
# This is a function for making a 3D visualization of the fish in 3D for a frame
# This function is useful because it returns a regular image array, not a matplotlib figure or something
# This makes it much easier to write in a movie

def make_3D_plot_of_frame_positions(f_positions):
    ''' Given the frame positions, return a 3D plot of the positions of this fish in
        array form
    '''
    # turn off interactive plots for now
    plt.ioff()

    # get the number of bps and numFish
    numFish = f_positions.shape[0]
    numBps = f_positions.shape[1]

    # make a figure and canvas
    fig = plt.figure(figsize=(15,15))
    fig.tight_layout(pad=0)
    canvas = FigureCanvas(fig)
    fig.add_subplot(111, projection='3d')
    ax = fig.gca()

    #loop over fish
    for f in range(numFish):

            
        # decide on the color
        if f == 0:
            c_real = [0,0,1,1]
            c_projection = [0,0,1,0.15]
        elif f == 1:
            c_real = [1,0,0,1]
            c_projection = [1,0,0,0.15]
        else:
            raise TypeError('Need to define more colors')

        # get the positions of the bps for this fish
        f_pos = f_positions[f]

        #  -- real -- #
        # plot each of the points
        ax.scatter(f_pos[0,0], f_pos[0,1], f_pos[0,2], c=c_real, marker='D', depthshade=False, s=40)
        ax.scatter(f_pos[1,0], f_pos[1,1], f_pos[1,2], c=c_real, marker='x', depthshade=False, s=40)
        ax.scatter(f_pos[2,0], f_pos[2,1], f_pos[2,2], c=c_real, marker='.', depthshade=False, s=40)
        # use plot3D to draw lines to join these points
        ax.plot3D(f_pos[:, 0], f_pos[:, 1], f_pos[:, 2], color=c_real, linewidth=2)

        # -- XY projection -- #
        # we want to set the Z coord of everypoint to 0
        # color is red with alpha layer
        ax.scatter(f_pos[0,0], f_pos[0,1], 0, c=c_projection, marker='D', depthshade=False, s=40)
        ax.scatter(f_pos[1,0], f_pos[1,1], 0, c=c_projection, marker='x', depthshade=False, s=40)
        ax.scatter(f_pos[2,0], f_pos[2,1], 0, c=c_projection, marker='.', depthshade=False, s=40)
        # use plot3D to draw lines to join these points
        ax.plot3D(f_pos[:, 0], f_pos[:, 1], np.zeros((numBps,)), c=c_projection, linewidth=2)

        # -- XZ projection -- #
        # we want to set the Y coord of everypoint to 0
        # color is red with alpha layer
        ax.scatter(f_pos[0,0], 0, f_pos[0,2], c=c_projection, marker='D', depthshade=False, s=40)
        ax.scatter(f_pos[1,0], 0, f_pos[1,2], c=c_projection, marker='x', depthshade=False, s=40)
        ax.scatter(f_pos[2,0], 0, f_pos[2,2], c=c_projection, marker='.', depthshade=False, s=40)
        # use plot3D to draw lines to join these points
        ax.plot3D(f_pos[:,0], np.zeros((numBps,)), f_pos[:,2], c=c_projection, linewidth=2)

        # -- YZ projection -- #
        # we want to set the X coord of everypoint to 0
        # color is red with alpha layer
        ax.scatter(0, f_pos[0,1], f_pos[0,2], c=c_projection, marker='D', depthshade=False, s=40)
        ax.scatter(0, f_pos[1,1], f_pos[1,2], c=c_projection, marker='x', depthshade=False, s=40)
        ax.scatter(0, f_pos[2,1], f_pos[2,2], c=c_projection, marker='.', depthshade=False, s=40)
        # use plot3D to draw lines to join these points
        ax.plot3D(np.zeros((numBps,)), f_pos[:,1], f_pos[:,2], c=c_projection, linewidth=2)


    # make the plot pretty
    ax.set_xlabel('X (cm)', fontsize=20)
    ax.set_ylabel('Y (cm)', fontsize=20)
    ax.set_zlabel('Z (cm)', fontsize=20)
    ax.set_xlim(0,40)
    ax.set_ylim(0,40)
    ax.set_zlim(0,40)
    ticks = [0, 10, 20, 30, 40]
    ax.set_xticks(ticks, minor=False)
    ax.set_yticks(ticks, minor=False)
    #ax.grid(False)

    # set the angle of viewing
    elevation_angle = 10
    azimuthal_angle = 45
    ax.view_init(elevation_angle, azimuthal_angle)

    # draw the canvas
    canvas.draw()

    # get the width and height of the figure, and turn them to type ints
    height, width = fig.get_size_inches() * fig.get_dpi()
    width = width.astype('int')
    height = height.astype('int')

    # convert the figure to an array of the correct size
    img = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(width, height, 3)

    # close the figure - we are done. You can also turn back on interactive plots now
    plt.close()
    plt.ion()

    # lastly remove some of the white space from around the frame
    # NOTE: IF YOU CHANGE FIGURE SIZE YOU WILL HAVE TO CHANGE THIS
    #       THIS IS A MANUAL WAY TO REMOVE WHITESPACE
    #img = img[325:1225, 250:1265]
    img = img[300:900, 150:1000]

    # return the image
    return img

# Displaying a movie

In [31]:
camIdx = 0
movPath = splitdata_paths_widget_list[camIdx].outputs[0]['data']['text/plain'][1:-1] + '.mp4'

In [36]:
video = Video.from_file(movPath)

In [37]:

video

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\xf95\xf1mdat\x00\x…